<a href="https://colab.research.google.com/github/Joongeun/MIT-Internship-2024/blob/main/Web_Scraping_for_Science_Feedback_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [ ]:
!pip install requests
!pip install BeautifulSoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import csv
import time
import pandas as pd
from pprint import pprint
import re

# Gathering All Headings of Claim/Article Reviews (No Need to run)

In [ ]:
pg_num = 1
claim, source, verdict, key_takeaway, review, headings = [], [], [], [], [], []
d = {}
for pg_num in range(1, 35): # Looping through pages 1-34
  url = f"https://science.feedback.org/reviews/?_topic=climate&_pagination={pg_num}/"
  html = requests.get(url)
  soup = BeautifulSoup(html.text, 'html.parser')
  # Extracts the claim & URL from the page
  info = soup.find_all("a", rel="bookmark")[:-3]
  # pprint(info)
  # Looping through each claim & URL
  for i in info:
    sub_url = i.attrs['href'] # Extracts the articl/claim review's URL
    html = requests.get(sub_url)
    soup = BeautifulSoup(html.text, 'html.parser')
    # print(soup)
    # Gets all text from "Review section" - ResultSet object with 1 element (all text) is returned (indexing works because it's similar to a list)
    info = soup.find_all("h2", class_="wp-block-heading")
    print(info)
    break
    for p in info:
      t = p.get_text()
      if t not in headings:
        d[t] = 1
        headings.append(t)
        # print("Heading:", t)
        # print("URL", sub_url)
      else:
        d[t] += 1

[<h2 class="has-exo-2-font-family has-heading-4-font-size wp-block-heading" style="text-transform:uppercase">Key takeaway</h2>, <h2 class="has-exo-2-font-family has-heading-3-font-size wp-block-heading" style="text-transform:uppercase">Reviewed content</h2>, <h2 class="has-exo-2-font-family has-heading-3-font-size wp-block-heading" style="text-transform:uppercase">Verdict detail</h2>, <h2 class="has-exo-2-font-family has-heading-3-font-size wp-block-heading" style="text-transform:uppercase">Full Claim</h2>, <h2 class="has-exo-2-font-family has-md-font-size wp-block-heading" style="text-transform:uppercase">Tags:</h2>, <h2 class="wp-block-heading has-exo-2-font-family has-heading-3-font-size" style="margin-bottom:var(--wp--preset--spacing--md);text-transform:uppercase">Share</h2>, <h2 class="wp-block-heading has-exo-2-font-family has-heading-3-font-size" style="margin-bottom:var(--wp--preset--spacing--md);text-transform:uppercase">Editor</h2>, <h2 class="has-exo-2-font-family has-xs-fon

In [ ]:
dict(sorted(d.items(), key=lambda item: item[1]))

{}

In [ ]:
# Expert analyses from all the new claim/article reviews will be under these unique headings
keep_headings = ['Key takeaway', 'Summary', 'Reviewed content', 'Verdict detail', 'Full Claim', 'Scientists’ Feedback',
                 'Review', 'Reviewers’ Feedback', 'Annotations', 'Reviewers’ Overall Feedback',
                 'Scientist’s Feedback', 'UPDATES', 'Featured Annotations', 'Reviewers’ Comments',
                 'GUEST COMMENTS', 'REVIEWERS’ OVERALL FEEDBACK', 'Featured Annotations\xa0']
# The "unique" list is the headers for just for 1 claim review
unique = ['CLAIM 1 (Inaccurate):', 'CLAIM 2 (Incorrect):']
# The "variations" list are repeats of the heading "Key takeaways" and "Summary"
variations = ['Key Take-aways', 'Key Takeaways', 'SUMMARY', 'Key Take-away']

In [ ]:
a = keep_headings+unique+variations
for i in a:
  print(i, d[i])

Key takeaway 172
Summary 32
Reviewed content 323
Verdict detail 170
Full Claim 172
Scientists’ Feedback 179
Review 26
Reviewers’ Feedback 3
Annotations 49
Reviewers’ Overall Feedback 17
Scientist’s Feedback 1
UPDATES 1
Featured Annotations 17
Reviewers’ Comments 1
GUEST COMMENTS 1
REVIEWERS’ OVERALL FEEDBACK 1
Featured Annotations  1
CLAIM 1 (Inaccurate): 1
CLAIM 2 (Incorrect): 1
Key Take-aways 49
Key Takeaways 2
SUMMARY 1
Key Take-away 3


# Final: Gathering All Claim & Article Reviews

Extracting all claim/article review URLs and verdicts (340 extracted; 28 of them have no verdict/credibility score, so only 312 will be used)

In [ ]:
# @title (Code to find all urls–no need to run)
pg_num = 1
url_list = []
for pg_num in range(1, 35): # Looping through pages 1-34 – skip the single post on pg 35
  url = f"https://science.feedback.org/reviews/?_topic=climate&_pagination={pg_num}/"
  html = requests.get(url)
  soup = BeautifulSoup(html.text, 'html.parser')
  # Extracts the claim & URL from the page–exclude the last 3 links because those are irrelevant articles (e.x: health) in the sidebar
  info = soup.find_all("a", rel="bookmark")[:-3]
  # pprint(info)
  # Looping through each claim & URL
  for i in info:
    sub_url = i.attrs['href'] # Extracts the articl/claim review's URL
    print(sub_url)
    url_list.append(sub_url)

print(url_list)
print(len(url_list))

In [33]:
#@title Initial url_list (No need to run)
url_list = ['https://science.feedback.org/review/pyramid-shaped-peaks-antarctica-naturally-carved-by-glaciers-not-constructed-by-ancient-civilization/', 'https://science.feedback.org/review/sea-levels-have-risen-for-over-100-years-despite-misleading-photos-social-media/', 'https://science.feedback.org/review/colorful-auroras-seen-may-2024-caused-by-mass-ejections-from-sun-not-haarp-experiments/', 'https://science.feedback.org/review/no-evidence-significant-influence-volcanoes-solar-variability-on-recent-climate-change-contrary-judith-curry-claims-prageru-video/', 'https://science.feedback.org/review/carbon-isotopes-do-not-show-humans-climate-impacts-too-small-notice-despite-the-daily-sceptic-inaccurate-claim/', 'https://science.feedback.org/review/review-climate-the-movie-the-cold-truth-reveals-numerous-well-known-misinformation-talking-points-inaccuracies/', 'https://science.feedback.org/review/data-shows-temperatures-rising-greenland-world-current-global-warming-driven-co2-not-solar-activity/', 'https://science.feedback.org/review/arctic-sea-ice-melting-over-past-decades-with-projections-summer-ice-free-conditions-mid-century/', 'https://science.feedback.org/review/optimal-atmospheric-co2-dinosaurs-plants-harmful-humans-current-concentration-higher-homo-sapiens-ever-experienced/', 'https://science.feedback.org/review/study-calls-people-grow-their-own-food-years-come-despite-finding-higher-carbon-footprint-compared-conventional-agriculture/', 'https://science.feedback.org/review/us-proposed-bills-hearings-dont-confirm-chemtrails-exist-not-geoengineering-strategy/', 'https://science.feedback.org/review/evidence-greenhouse-gasses-cause-global-warming-denied-willie-soon-tucker-carlson-interview-mass-social-media-climate-misinformation/', 'https://science.feedback.org/review/eu-not-passing-law-seize-scrap-cars-under-climate-agenda-contrary-peter-sweden-claim/', 'https://science.feedback.org/review/contrary-widespread-misrepresentation-new-study-finds-extremely-minimal-impact-human-breathing-climate/', 'https://science.feedback.org/review/claim-cow-burps-not-important-contributor-climate-change-ignores-full-range-greenhouse-gas-emissions-beef-production/', 'https://science.feedback.org/review/uncertainties-about-future-climate-change-1970s-does-not-invalidate-today-evidence-reality-global-warming/', 'https://science.feedback.org/review/water-vapor-greenhouse-gas-not-major-driver-global-warming/', 'https://science.feedback.org/review/video-ian-plimer-incorrectly-states-human-co2-emissions-not-responsible-for-increased-atmospheric-co2-concentrations-global-warming/', 'https://science.feedback.org/review/global-warming-matches-model-predictions-well-misconstrued-claims-new-photosynthesis-study/', 'https://science.feedback.org/review/wall-street-journal-questions-decades-scientific-evidence-demonstrating-elevated-atmospheric-co2-causes-global-warming/', 'https://science.feedback.org/review/claim-current-climate-change-explained-natural-cycles-volcanic-activity-does-not-have-scientific-support/', 'https://science.feedback.org/review/clintel-group-inaccurately-represents-climate-science-declaration-no-climate-emergency-once-again/', 'https://science.feedback.org/review/three-decades-chemtrail-haarp-geoengineering-conspiracy-theory-remains-popular-despite-lack-evidence-no-scientific-basis/', 'https://science.feedback.org/review/magnetic-poles-reversals-not-cataclysmic-events-adam-and-eve-theory-jimmy-corsetti-joe-rogan-podcast/', 'https://science.feedback.org/review/headline-temperatures-on-earth-are-increasing-and-the-rise-is-drastically-outpacing-previous-natural-changes-in-the-planets-climate/', 'https://science.feedback.org/review/antarctic-ice-coverage-not-increasing-record-levels-nor-continent-getting-colder-contrary-to-curry-easterbrook-cfact-movie/', 'https://science.feedback.org/review/strands-cloud-behind-aeroplanes-trails-condensed-water-contrails-not-chemtrails-some-believe/', 'https://science.feedback.org/review/pre-human-changes-climate-not-negate-contemporary-human-influence-modern-warming/', 'https://science.feedback.org/review/link-between-co2-earth-temperature-well-established-despite-claims-fox-news-tom-harris/', 'https://science.feedback.org/review/scientists-found-modern-climate-change-caused-entirely-by-human-activity-contrary-recent-speculation-joe-rogan-experience/', 'https://science.feedback.org/review/the-epoch-times-prints-range-of-inaccurate-misleading-claims-climate-changes-impacts-causes-patrick-moore/', 'https://science.feedback.org/review/scientific-evidence-indicates-modern-warming-driven-by-co2-contrary-to-richard-lindzen/', 'https://science.feedback.org/review/letter-there-is-no-climate-emergency-repeats-inaccurate-claims-about-climate-science-daily-sceptic-toby-young/', 'https://science.feedback.org/review/carbon-dioxide-emissions-humans-major-driver-climate-change-jerome-corsi-claims-cbn-news/', 'https://science.feedback.org/review/the-sun-isnt-responsible-for-current-climate-change-contrary-claims-suspicious0bservers-ben-davidson/', 'https://science.feedback.org/review/warming-earths-surface-oceans-continues-apace-contrary-to-claims-daily-sceptic-chris-morrison/', 'https://science.feedback.org/review/earths-surface-warm-due-greenhouse-gas-christopher-monckton-watts-up-with-that/', 'https://science.feedback.org/review/robust-scientific-evidence-supports-human-activity-drives-global-warming-contrary-to-claims-co2-coalition-blog-post-andy-may/', 'https://science.feedback.org/review/more-than-75-percent-amazon-rainforest-losing-resilience-the-washington-post-sarah-kaplan/', 'https://science.feedback.org/review/wall-street-journal-steven-koonin-publishes-misleading-claims-climate-change-influences-greenland-ice-melt/', 'https://science.feedback.org/review/prageru-video-on-climate-change-repeats-a-range-of-misleading-claims-by-steven-koonin/', 'https://science.feedback.org/review/coral-cover-great-barrier-reef-improved-2021-but-doesnt-mean-reef-growing-quickly-contrary-to-daily-wire/', 'https://science.feedback.org/review/rain-fell-on-greenland-summit-for-the-first-time-on-record-as-accurately-reported-by-cnn/', 'https://science.feedback.org/review/a-2014-study-showed-that-the-us-corn-belt-is-one-of-the-biggest-primary-producers-on-earth-in-july-but-didnt-show-that-it-produces-more-oxygen-than-the-amazon/', 'https://science.feedback.org/review/global-warming-contributes-to-increased-heat-related-mortality-contrary-to-bjorn-lomborgs-unsupported-claims-that-climate-change-is-saving-hundreds-of-thousands-of-lives-each-year/', 'https://science.feedback.org/review/sea-temperatures-around-the-great-barrier-reef-have-increased-over-the-last-150-years-contrary-to-claim-at-wattsupwiththat/', 'https://science.feedback.org/review/solar-forcing-is-not-the-main-cause-of-current-global-warming-contrary-to-claim-by-alex-newman-in-the-epoch-times/', 'https://science.feedback.org/review/despite-2021-being-a-good-year-for-coral-health-coral-in-the-great-barrier-reef-has-declined-over-the-past-decade-and-is-threatened-by-climate-change-contrary-to-claims-by-peter-ridd/', 'https://science.feedback.org/review/youtube-video-falsely-claims-that-climate-models-systematically-over-estimated-warming-and-that-we-are-heading-towards-an-ice-age/', 'https://science.feedback.org/review/gregory-wrightstone-article-in-the-washington-times-presents-list-of-false-and-misleading-statements-about-the-impacts-of-co2-and-climate-change-co2-coalition/', 'https://science.feedback.org/review/wall-street-journal-article-repeats-multiple-incorrect-and-misleading-claims-made-in-steven-koonins-new-book-unsettled-steven-koonin/', 'https://science.feedback.org/review/bidens-climate-plan-doesnt-say-anything-about-limiting-meat-consumption-contrary-to-daily-mail-fox-news-claim/', 'https://science.feedback.org/review/video-promoted-by-john-stossel-for-earth-day-relies-on-incorrect-and-misleading-claims-about-climate-change/', 'https://science.feedback.org/review/a-third-of-antarctic-ice-shelf-risks-collapse-at-4c-above-pre-industrial-levels-as-accurately-described-in-cnn-article/', 'https://science.feedback.org/review/in-viral-turning-point-usa-video-candace-owens-and-charlie-kirk-falsely-claim-there-is-no-evidence-of-global-warming-and-scientists-dont-know-the-cause/', 'https://science.feedback.org/review/deserts-cool-the-planet-by-reflecting-solar-radiation-to-space-vegetated-areas-have-an-overall-warming-effect-so-planting-trees-in-deserts-doesnt-necessarily-cool-the-planet-the-guardian/', 'https://science.feedback.org/review/solar-geoengineering-isnt-happening-or-damaging-the-planet-aircraft-contrails-are-formed-by-water-vapor-not-chemicals/', 'https://science.feedback.org/review/destruction-of-the-amazon-forest-very-likely-contributes-to-global-warming-as-accurately-described-in-the-national-geographic-article/', 'https://science.feedback.org/review/atlantic-ocean-circulation-system-is-slowing-down-as-accurately-described-in-the-new-york-times-article/', 'https://science.feedback.org/review/scientists-didnt-announce-impending-environmental-catastrophes-every-decade-since-the-1970s/', 'https://science.feedback.org/review/the-scientific-research-facility-haarp-cannot-create-natural-disasters-or-influence-human-thoughts-and-movements-contrary-to-online-claims/', 'https://science.feedback.org/review/breitbart-article-makes-numerous-false-claims-about-the-impacts-of-climate-change-based-on-global-warming-policy-foundation-post-delingpole-goklany/', 'https://science.feedback.org/review/noaa-shows-clear-global-warming-trends-over-the-past-58-years-based-on-radiosonde-data/', 'https://science.feedback.org/review/prageru-post-by-william-happer-uses-flawed-reasoning-to-claim-that-climate-models-always-fail/', 'https://science.feedback.org/review/video-interview-of-ian-plimer-at-sky-news-falsely-claims-that-a-new-study-announces-an-incoming-ice-age-partly-based-on-an-incorrect-daily-mail-headline/', 'https://science.feedback.org/review/rates-of-global-sea-level-rise-have-accelerated-since-1900-contrary-to-bloggers-claims/', 'https://science.feedback.org/review/scientific-studies-established-clear-links-between-human-caused-increased-in-atmospheric-co2-and-global-warming-patrick-moore/', 'https://science.feedback.org/review/the-clothing-industry-produces-3-to-10-of-global-greenhouse-gas-emissions-as-accurately-claimed-in-patagonia-post/', 'https://science.feedback.org/review/brush-rake-attachments-are-primarily-used-to-pile-up-heavy-forest-debris-as-part-of-active-thinning-harvesting-and-removal-projects/', 'https://science.feedback.org/review/guardian-article-on-arctic-methane-emissions-lacks-important-context-jonathan-watts/', 'https://science.feedback.org/review/sea-ice-loss-due-to-climate-change-is-the-biggest-threat-to-polar-bear-survival-and-has-already-led-to-declines-in-some-polar-bear-subpopulations-contrary-to-climate-realism-video-m/', 'https://science.feedback.org/review/climate-change-can-make-it-harder-for-the-amazon-rainforest-to-grow-back-from-deforestation-but-that-does-not-mean-40-of-it-will-now-turn-into-a-savanna/', 'https://science.feedback.org/review/article-by-the-daily-caller-oversimplifies-drivers-of-wildfires-and-downplays-role-of-climate-change-chris-white/', 'https://science.feedback.org/review/climate-change-forest-management-and-several-other-causes-contribute-to-wildfire-severity-and-total-area-burned-in-the-western-united-states/', 'https://science.feedback.org/review/western-us-wildfires-are-not-the-result-of-widespread-arson/', 'https://science.feedback.org/review/planting-trees-can-help-mitigate-some-aspects-of-climate-change-but-it-cannot-solve-all-environmental-crises/', 'https://science.feedback.org/review/article-by-cnn-exaggerates-studys-implications-for-future-greenland-ice-loss/', 'https://science.feedback.org/review/the-long-term-survival-of-polar-bears-is-threatened-by-loss-of-sea-ice-due-to-global-warming-new-study-confirms-henry-fountain/', 'https://science.feedback.org/review/greenhouse-gases-cause-global-warming-by-trapping-infra-red-radiations-not-by-causing-more-holes-in-the-ozone-layer/', 'https://science.feedback.org/review/article-by-michael-shellenberger-mixes-accurate-and-inaccurate-claims-in-support-of-a-misleading-and-overly-simplistic-argumentation-about-climate-change/', 'https://science.feedback.org/review/study-evaluates-natural-and-human-causes-of-recent-rapid-warming-rate-at-the-south-pole/', 'https://science.feedback.org/review/climate-scientists-agree-that-human-caused-greenhouse-gas-emissions-are-primarily-responsible-for-climate-change-contrary-to-claims-in-clear-energy-alliance-video/', 'https://science.feedback.org/review/article-in-the-guardian-misleads-readers-about-sensitivity-of-climate-models-by-narrowly-focusing-on-single-study-jonathan-watts/', 'https://science.feedback.org/review/video-from-prageru-makes-several-incorrect-and-misleading-claims-about-climate-change-richard-lindzen/', 'https://science.feedback.org/review/low-solar-activity-has-little-effect-on-earths-climate-contrary-to-claim-in-the-sun/', 'https://science.feedback.org/review/the-global-polar-bear-population-is-threatened-by-loss-of-sea-ice-contrary-to-pragerus-video-claim/', 'https://science.feedback.org/review/article-in-business-insider-accurately-describes-results-from-a-study-estimating-up-to-3-billion-people-could-live-in-much-warmer-temperatures-by-2070/', 'https://science.feedback.org/review/diesel-cars-are-a-major-source-of-no2-emissions-in-european-cities-contrary-to-online-claim/', 'https://science.feedback.org/review/claim-that-co2-is-cooling-the-planet-is-misleading-anthropogenic-co2-emissions-overall-cause-global-warming/', 'https://science.feedback.org/review/human-activities-have-dramatically-increased-atmospheric-co2-levels-causing-imbalances-in-the-global-carbon-cycle/', 'https://science.feedback.org/review/human-induced-increases-in-greenhouse-gases-are-the-primary-driver-of-global-warming-contrary-to-claims-in-cfact-article/', 'https://science.feedback.org/review/scientists-observe-most-widespread-bleaching-event-ever-recorded-at-the-great-barrier-reef/', 'https://science.feedback.org/review/the-antarctic-ozone-layer-is-recovering-but-the-independent-claim-fails-to-grasp-significance-of-study-it-relies-on/', 'https://science.feedback.org/review/deforestation-can-facilitate-the-emergence-and-spread-of-some-infectious-diseases/', 'https://science.feedback.org/review/global-data-contradict-claim-of-no-acceleration-in-sea-level-rise/', 'https://science.feedback.org/review/global-warming-and-the-covid-19-pandemic-are-unrelated-issues-one-will-not-protect-us-from-the-other-life-site-news/', 'https://science.feedback.org/review/temperature-trends-in-the-u-s-are-consistent-with-warming-around-the-world-contrary-to-electroverse-claim-cap-allon/', 'https://science.feedback.org/review/co2-can-increase-plant-growth-in-greenhouses-while-also-negatively-affecting-ecosystems-and-human-societies-mike-adams-natural-news/', 'https://science.feedback.org/review/electroverse-article-incorrectly-claims-the-sun-is-behind-climate-change/', 'https://science.feedback.org/review/sea-levels-rose-faster-in-the-past-century-than-in-previous-time-periods/', 'https://science.feedback.org/review/article-in-the-sun-misrepresents-antarctic-discovery-and-misplaces-it-on-map-charlotte-edwards/', 'https://science.feedback.org/review/data-from-glaciers-around-the-world-show-most-are-shrinking-as-a-consequence-of-global-warming/', 'https://science.feedback.org/review/current-climate-warming-is-rapid-and-occurring-on-a-global-scale-unlike-past-periods-of-regional-climate-fluctuations/', 'https://science.feedback.org/review/sea-levels-have-risen-throughout-the-20th-century-contrary-to-claim-in-online-article/', 'https://science.feedback.org/review/noaa-does-not-fraudulently-adjust-data-to-create-the-illusion-of-a-global-warming-trend-investors/', 'https://science.feedback.org/review/co2-emissions-from-human-activities-have-imbalanced-the-atmospheric-carbon-budget-significantly-contributing-to-climate-change-contrary-to-online-claim-issues-insights/', 'https://science.feedback.org/review/claims-of-a-coming-30-year-mini-ice-age-are-not-supported-by-science-the-sun/', 'https://science.feedback.org/review/studies-do-not-support-blogs-claim-that-volcanic-activity-is-causing-melting-of-west-antarctic-glaciers/', 'https://science.feedback.org/review/simple-measurements-demonstrate-that-co2-is-a-greenhouse-gas-tim-ball/', 'https://science.feedback.org/review/facebook-meme-incorrectly-claims-past-climate-variations-contradict-human-influence-on-current-climate/', 'https://science.feedback.org/review/climate-change-bushfires-australia-breitbart-newspunch/', 'https://science.feedback.org/review/volcanic-co2-emissions-are-known-to-be-much-smaller-than-human-caused-emissions-james-edward-kamis/', 'https://science.feedback.org/review/article-falsely-attributes-invented-quotes-to-james-hansen-dmitry-baxter/', 'https://science.feedback.org/review/ice-cube-meme-misrepresents-physics-of-sea-level-rise-to-claim-melting-ice-has-no-effect/', 'https://science.feedback.org/review/1977-coming-ice-age-time-magazine-cover-is-a-fake/', 'https://science.feedback.org/review/abc-article-effectively-illustrates-important-climate-trends-for-australian-readers-tim-leslie-joshua-byrd-nathan-hoad/', 'https://science.feedback.org/review/ian-plimer-op-ed-in-the-australian-again-presents-long-list-of-false-claims-about-climate/', 'https://science.feedback.org/review/bedrock-heat-flow-studies-made-no-claim-about-human-caused-melting-climate-change-dispatch-james-edward-kamis/', 'https://science.feedback.org/review/new-york-times-op-ed-claiming-scientists-underestimated-climate-change-lacks-supporting-evidence-eugene-linden/', 'https://science.feedback.org/review/claim-of-a-coming-ice-age-misrepresents-the-study-it-relies-on/', 'https://science.feedback.org/review/human-activities-release-far-more-carbon-dioxide-than-volcanoes-do/', 'https://science.feedback.org/review/letter-to-un-was-not-signed-by-500-experts-on-climate-science-breitbart/', 'https://science.feedback.org/review/photo-meme-of-sydney-harbour-incorrectly-claims-no-sea-level-rise-has-occurred/', 'https://science.feedback.org/review/telegraph-article-misleads-with-false-balance-mixing-in-unsupported-and-inaccurate-claims-sarah-knapton/', 'https://science.feedback.org/review/letter-signed-by-500-scientists-relies-on-inaccurate-claims-about-climate-science/', 'https://science.feedback.org/review/2c-not-known-point-of-no-return-as-jonathan-franzen-claims-new-yorker/', 'https://science.feedback.org/review/there-is-overwhelming-evidence-that-climate-change-is-human-caused-townhall/', 'https://science.feedback.org/review/claim-that-the-earth-has-cooled-since-medieval-or-roman-times-is-false-marc-morano/', 'https://science.feedback.org/review/claim-of-no-us-warming-since-2005-is-directly-contradicted-by-the-data-it-is-based-on/', 'https://science.feedback.org/review/earths-orbit-cannot-explain-modern-climate-change/', 'https://science.feedback.org/review/washington-examiner-op-ed-cherry-picks-data-to-mislead-readers-about-climate-models-patrick-michaels-caleb-stewart-rossiter/', 'https://science.feedback.org/review/prediction-extinction-rebellion-climate-change-will-kill-6-billion-people-unsupported-roger-hallam-bbc/', 'https://science.feedback.org/review/rick-perrys-claim-that-us-is-leading-the-world-in-emission-cuts-is-misleading-fox-news/', 'https://science.feedback.org/review/its-not-true-that-co2-only-increases-after-temperature-rises-william-happer-ice-age-now/', 'https://science.feedback.org/review/claim-that-cosmic-rays-are-a-crucial-player-for-current-climate-change-is-unsupported-nir-shaviv-electroverse-global-warming-policy-foundation/', 'https://science.feedback.org/review/yet-another-express-article-falsely-warns-of-a-solar-induced-ice-age-sean-martin/', 'https://science.feedback.org/review/research-does-not-show-a-medieval-warm-period-warmer-than-the-present-day/', 'https://science.feedback.org/review/hosted-by-former-australian-senator-tony-heller-repeats-false-claim-that-scientists-fake-the-warming-trend/', 'https://science.feedback.org/review/nbc-news-story-accurately-covers-research-on-two-millennia-of-climate-history-jaclyn-jeffrey-wilensky/', 'https://science.feedback.org/review/the-earth-was-not-warmer-in-medieval-times-town-hall-gregory-rummo/', 'https://science.feedback.org/review/sky-news-australia-interview-falsely-claims-that-global-cooling-is-coming-soon/', 'https://science.feedback.org/review/non-peer-reviewed-manuscript-falsely-claims-natural-cloud-changes-can-explain-global-warming/', 'https://science.feedback.org/review/on-fox-news-patrick-michaels-falsely-claims-humans-are-only-responsible-for-half-of-global-warming/', 'https://science.feedback.org/review/popular-article-in-the-independent-accurately-summarizes-june-heat-in-europe-conrad-duncan/', 'https://science.feedback.org/review/nasa-did-not-create-global-warming-by-manipulating-data-tony-heller-steven-goddard/', 'https://science.feedback.org/review/claim-that-electric-vehicles-cause-more-carbon-emissions-than-diesel-misrepresents-reality/', 'https://science.feedback.org/review/investors-business-daily-editorial-doesnt-support-claims-about-un-statements/', 'https://science.feedback.org/review/financial-post-commentary-misrepresents-scientific-understanding-of-weather-extremes-ross-mckitrick/', 'https://science.feedback.org/review/iflscience-story-on-speculative-report-provides-little-scientific-context-james-felton/', 'https://science.feedback.org/review/the-australian-commentary-by-ian-plimer-relies-on-false-claims-to-make-its-case/', 'https://science.feedback.org/review/its-true-that-the-current-carbon-dioxide-level-is-higher-than-any-time-in-human-existence/', 'https://science.feedback.org/review/video-falsely-claiming-to-be-barron-trump-also-falsely-claims-humans-arent-causing-climate-change/', 'https://science.feedback.org/review/there-are-clear-evidence-for-climate-change-contrary-to-claims-by-john-coleman/', 'https://science.feedback.org/review/claim-in-the-express-that-low-solar-activity-is-bringing-cold-weather-is-false/', 'https://science.feedback.org/review/breitbart-article-baselessly-claims-a-study-of-past-climate-invalidates-human-caused-climate-change-john-nolte/', 'https://science.feedback.org/review/blaze-article-badly-misrepresents-study-of-greenlands-jakobshavn-glacier/', 'https://science.feedback.org/review/caleb-rossiter-falsely-claims-that-climate-models-are-running-very-hot/', 'https://science.feedback.org/review/senator-sanders-claim-that-climate-change-is-making-tornadoes-worse-isnt-supported-by-published-research/', 'https://science.feedback.org/review/western-journal-op-ed-deceives-readers-with-completely-unsupported-claims-jay-lehr-tom-harris/', 'https://science.feedback.org/review/claim-about-climate-impacts-of-cows-vs-cars-needs-a-little-explanation-financial-times/', 'https://science.feedback.org/review/new-york-times-coverage-of-ipcc-report-clearly-presents-conclusions-coral-davenport/', 'https://science.feedback.org/review/guardian-story-on-climate-impacts-of-diet-gets-mixed-reviews-from-scientists-damian-carrington/', 'https://science.feedback.org/review/story-on-congressmans-incorrect-claims-about-sea-level-rise-could-have-corrected-them-more-explicitly/', 'https://science.feedback.org/review/bbc-article-on-ipcc-report-is-mostly-accurate-but-could-use-some-clarification/', 'https://science.feedback.org/review/cnn-accurately-covers-latest-ipcc-report-brandon-miller/', 'https://science.feedback.org/review/usa-today-story-updates-readers-on-trend-in-monthly-global-temperatures-doyle-rice/', 'https://science.feedback.org/review/washington-post-article-accurately-discusses-warm-arctic-weather-event-jason-samenow/', 'https://science.feedback.org/review/friends-of-science-video-promoted-by-youtube-presents-long-list-of-climate-myths-steve-goreham/', 'https://science.feedback.org/review/popular-story-fox2news-accurately-describes-2017-global-temperature-sea-ice-extent-cnn/', 'https://science.feedback.org/review/metros-claims-of-coming-mini-ice-age-have-no-basis-in-reality/', 'https://science.feedback.org/review/washington-post-accurately-describes-ocean-warming-study-with-potential-implications-for-future-carbon-budget-chris-mooney-brady-dennis/', 'https://science.feedback.org/review/guardian-story-accurately-describes-study-on-environmental-impacts-of-our-food-system-damian-carrington/', 'https://science.feedback.org/review/on-bbc-newsnight-myron-ebell-falsely-claims-climate-models-exaggerate-warming/', 'https://science.feedback.org/review/guardian-conveys-statement-scientists-stressing-role-of-deforestation-climate-change-oliver-milman/', 'https://science.feedback.org/review/usa-today-op-ed-ignores-evidence-to-claim-climate-change-had-no-role-in-hurricane-florence-roy-spencer/', 'https://science.feedback.org/review/guardian-story-accurately-covers-sea-ice-event-but-makes-unsupported-connection-weather-patterns-gulf-stream-jonathan-watts/', 'https://science.feedback.org/review/ian-plimer-wrongly-claims-that-carbon-dioxide-emissions-do-not-cause-climate-change/', 'https://science.feedback.org/review/ian-plimers-op-ed-incorrectly-claims-human-caused-emissions-are-short-lived/', 'https://science.feedback.org/review/op-ed-in-the-australian-gets-nearly-every-fact-wrong-ian-plimer/', 'https://science.feedback.org/review/guardian-potential-future-hothouse-climate-generally-accurate-but-misstates-details-jonathan-watts/', 'https://science.feedback.org/review/president-trumps-claim-that-water-supply-policy-has-worsened-california-wildfires-is-baseless/', 'https://science.feedback.org/review/washington-post-story-puts-recent-weather-extremes-in-accurate-climate-change-context-joel-achenbach-angela-fritz/', 'https://science.feedback.org/review/npr-story-accurately-describes-ecological-consequences-of-altered-spring-timings-in-a-warming-climate-nathan-rott/', 'https://science.feedback.org/review/fred-singer-incorrectly-claims-sea-level-rise-is-not-caused-by-climate-change/', 'https://science.feedback.org/review/usa-today-paleoclimate-study-mostly-accurate-lacks-clarity-doyle-rice/', 'https://science.feedback.org/review/scorching-earth-global-warming-blame-all-time-heat-records-worldwide-daily-mail-joe-pinkstone/', 'https://science.feedback.org/review/financial-post-commentary-misleads-warming-effect-greenhouse-gas-emissions-cherry-picking-studies-ross-mckitrick/', 'https://science.feedback.org/review/washington-post-article-accurately-describes-latest-estimate-accelerating-antarctic-ice-loss-chris-mooney/', 'https://science.feedback.org/review/national-geographic-hurricanes-moving-slower-global-warming-craig-welch/', 'https://science.feedback.org/review/new-york-times-story-accurately-describes-rio-grandes-climate-context-henry-fountain/', 'https://science.feedback.org/review/wall-street-journal-commentary-grossly-misleads-readers-about-science-of-sea-level-rise-fred-singer/', 'https://science.feedback.org/review/business-insider-highlights-health-impacts-climate-change-aspects-are-misleading-kevin-loria/', 'https://science.feedback.org/review/the-australian-coverage-great-barrier-reef-perception-scientists-divided-global-warming-graham-lloyd/', 'https://science.feedback.org/review/investors-business-daily-editorial-misrepresents-study-to-claim-plants-will-prevent-dangerous-climate-change/', 'https://science.feedback.org/review/rush-limbaugh-falsely-claims-there-is-no-evidence-of-human-caused-global-warming/', 'https://science.feedback.org/review/washington-post-accurately-covers-permafrost-study-albeit-under-a-somewhat-sensational-headline-chris-mooney/', 'https://science.feedback.org/review/financial-post-publishes-misleading-opinion-misrepresents-science-polar-bears-peril-susan-crockford/', 'https://science.feedback.org/review/breitbart-repeats-bloggers-unsupported-claim-noaa-manipulates-data-exaggerate-warming/', 'https://science.feedback.org/review/satellite-measurements-accelerating-sea-level-rise-cnn-accurately-reports-brandon-miller/', 'https://science.feedback.org/review/epas-scott-pruitt-incorrectly-suggests-climate-change-might-not-bad-thing/', 'https://science.feedback.org/review/president-trumps-claim-growing-ice-not-reflect-reality/', 'https://science.feedback.org/review/national-geographic-video-starving-polar-bear-should-have-clarified-uncertain-link-climate-change-sarah-gibbens/', 'https://science.feedback.org/review/atlantic-climate-implications-reduced-beef-consumption-provide-clearer-context-james-hamblin/', 'https://science.feedback.org/review/new-york-times-iceberg-larsen-c-size-delaware-jugal-patel-justin-gillis/', 'https://science.feedback.org/review/new-york-times-accurately-covers-2017-record-low-arctic-winter-sea-ice-extent-henry-fountain/', 'https://science.feedback.org/review/conservative-tribune-post-falsely-claims-cancelled-arctic-research-cruise-is-evidence-against-climate-change-benjamin-arie/', 'https://science.feedback.org/review/iflscience-story-florida-sea-level-rise-somewhat-unclear-generally-correct/', 'https://science.feedback.org/review/new-york-times-news-coverage-2016-global-temperature-data-accurate-summary-justin-gillis/', 'https://science.feedback.org/review/futurism-story-great-barrier-reef-compromised-sensational-headline/', 'https://science.feedback.org/review/breitbart-california-fires-misleads-omitting-explanation-climates-influence-joel-pollak/', 'https://science.feedback.org/review/daily-caller-uncritically-reports-misleading-satellite-temperature-study-michael-bastasch/', 'https://science.feedback.org/review/prof-john-christy-incorrectly-claims-to-show-climate-models-are-too-sensitive-to-carbon-dioxide/', 'https://science.feedback.org/review/antarctica-doomsday-glaciers-could-flood-coastal-cities-grist-eric-holthaus/', 'https://science.feedback.org/review/guardian-sea-level-rise-cities-fails-make-timescale-clear/', 'https://science.feedback.org/review/independent-makes-giant-leap-global-warming-worse-than-thought-single-study-andrew-griffin/', 'https://science.feedback.org/review/commentary-australian-ignores-evidence-misrepresents-research-falsely-claiming-humans-not-responsible-climate-change-ian-plimer/', 'https://science.feedback.org/review/bbc-article-ocean-acidification-accurate-brief-roger-harrabin/', 'https://science.feedback.org/review/former-prime-minister-tony-abbott-wrongly-claims-australia-warmed-far-less-data-show/', 'https://science.feedback.org/review/new-york-times-straightforward-answers-common-climate-questions-accurate-justin-gillis/', 'https://science.feedback.org/review/daily-wire-article-incorrectly-claims-climate-models-have-overestimated-warming/', 'https://science.feedback.org/review/daily-wire-article-misunderstands-study-carbon-budget-along-fox-news-telegraph-daily-mail-breitbart-james-barrett/', 'https://science.feedback.org/review/global-warming-fire-suppression-practices-boost-wildfires-us-west-correctly-reported-the-atlantic-robinson-meyer/', 'https://science.feedback.org/review/incorrect-claim-global-warming-mostly-natural-based-on-study-that-cant-support-conclusion-jennifer-marohasy-john-abbot/', 'https://science.feedback.org/review/climate-change-likely-helped-increase-hurricane-harveys-rainfall-early-say-exactly-much/', 'https://science.feedback.org/review/atlantic-accurately-explores-climate-context-tropical-storm-harvey-robinson-meyer/', 'https://science.feedback.org/review/new-york-times-accurately-assesses-the-state-of-alaskan-permafrost-henry-fountain/', 'https://science.feedback.org/review/extreme-weather-events-clearly-becoming-common-contrast-lord-lawsons-claim/', 'https://science.feedback.org/review/lord-lawson-falsely-claims-global-temperatures-declined-last-decade/', 'https://science.feedback.org/review/forbes-article-accurately-describes-research-atlantic-ocean-circulation-weakening-headline-goes-farther-trevor-nace/', 'https://science.feedback.org/review/wall-street-journal-op-ed-economic-consequences-climate-change-found-naive-scientists-david-henderson-john-cochrane/', 'https://science.feedback.org/review/2017-track-among-hottest-year-recorded-scientists-not-surprised-thinkprogress-article-suggests-joe-romm/', 'https://science.feedback.org/review/scientists-explain-what-new-york-magazine-article-on-the-uninhabitable-earth-gets-wrong-david-wallace-wells/', 'https://science.feedback.org/review/breitbart-article-falsely-claims-measured-global-warming-fabricated-james-delingpole/', 'https://science.feedback.org/review/earth-is-not-at-risk-of-becoming-a-hothouse-like-venus-as-stephen-hawking-claimed-bbc/', 'https://science.feedback.org/review/the-atlantic-accurately-reports-study-economic-impacts-continued-climate-change-us-robinson-meyer/', 'https://science.feedback.org/review/new-york-times-story-highlights-the-growing-number-of-extremely-hot-days-in-a-warming-world-brad-plumer-n-popovich/', 'https://science.feedback.org/review/energy-secretary-rick-perry-incorrectly-claims-co2-not-primary-cause-climate-change/', 'https://science.feedback.org/review/breitbart-misrepresents-research-58-scientific-papers-falsely-claim-disprove-human-caused-global-warming-james-delingpole/', 'https://science.feedback.org/review/breitbart-falsely-claims-58-studies-refute-human-caused-global-warming/', 'https://science.feedback.org/review/playing-semantics-misleading-breitbart-article-downplays-us-contribution-to-climate-change/', 'https://science.feedback.org/review/paris-agreement-op-ed-us-senator-ted-cruz-misrepresents-costs-benefits-reducing-greenhouse-gas-emissions/', 'https://science.feedback.org/review/natural-variability-can-not-explain-modern-global-warming-heartland-institute-report-claims/', 'https://science.feedback.org/review/lull-solar-activity-little-effect-global-temperatures-claims-global-cooling-not-based-science/', 'https://science.feedback.org/review/global-warming-not-ceased-heartland-institute-report-falsely-claims/', 'https://science.feedback.org/review/co2-greenhouse-gas-caused-warming-past-climate-changes-notably-feedback-amplifying-factors/', 'https://science.feedback.org/review/new-york-times-series-accurately-describes-research-antarctic-ice-sheets-sea-level-rise-justin-gillis/', 'https://science.feedback.org/review/heartland-institute-reports-claim-climate-models-sensitive-co2-not-reflect-evidence/', 'https://science.feedback.org/review/alaskas-vicious-cycle-warming-tundra-spews-co2-speeding-up-warming-joe-romm-think-progress/', 'https://science.feedback.org/review/heartland-institute-report-incorrectly-claims-no-evidence-human-impacts-melting-ice/', 'https://science.feedback.org/review/global-quackery-earth-not-warmed-past-19-years-new-study-finds-joseph-curl-the-daily-wire/', 'https://science.feedback.org/review/another-arctic-ice-panic-world-temperatures-plummet-the-telegraph-christopher-booker/', 'https://science.feedback.org/review/global-sea-level-rise-accelerating-despite-heartland-institute-reports-claims-otherwise/', 'https://science.feedback.org/review/the-sun-cannot-explain-recent-global-warming-contrary-to-what-heartland-institute-report-claims/', 'https://science.feedback.org/review/climate-sensitivity-estimate-given-in-heartland-institutes-report-is-misleading/', 'https://science.feedback.org/review/cnn-interview-william-happer-incorrectly-claims-temperatures-dont-match-climate-model-projections/', 'https://science.feedback.org/review/cnn-interview-william-happer-misleads-impact-rising-carbon-dioxide-plant-life/', 'https://science.feedback.org/review/bloomberg-accurately-describes-rapidly-changing-arctic-eric-roston-blacki-migliozzi/', 'https://science.feedback.org/review/climate-context-for-a-wet-winter-in-drought-stricken-california-in-the-la-times-rong-gong-lin-ii-paige-st-john-los-angeles-times/', 'https://science.feedback.org/review/sea-level-rise-already-practical-concern-miami-bbc-story-accurately-explores-amanda-ruggeri-bbc/', 'https://science.feedback.org/review/one-of-the-most-troubling-ideas-about-climate-change-just-found-new-evidence-in-its-favor-chris-mooney-washington-post/', 'https://science.feedback.org/review/analysis-record-breaking-climate-change-pushes-world-uncharted-territory-damian-carrington-guardian/', 'https://science.feedback.org/review/great-barrier-reef-coral-climate-change-dieoff-the-new-york-times-damien-cave-justin-gillis/', 'https://science.feedback.org/review/boston-globe-column-claims-climate-models-consistently-failed-based-no-evidence/', 'https://science.feedback.org/review/why-are-climate-change-models-so-flawed-because-climate-science-is-so-incomplete-jeff-jacoby-boston-globe/', 'https://science.feedback.org/review/claim-new-scientist-humans-responsible-100-warming-mostly-correct/', 'https://science.feedback.org/review/science-shows-humans-primarily-responsible-climate-change-counter-epa-administrator-scott-pruitts-claim/', 'https://science.feedback.org/review/scientists-we-know-what-really-causes-climate-james-barrett-the-daily-wire/', 'https://science.feedback.org/review/heres-why-its-so-frickin-hot-right-now-andrew-freedman-mashable/', 'https://science.feedback.org/review/worlds-coral-reefs-severely-threatened-climate-change-human-impacts-abc-story-notes/', 'https://science.feedback.org/review/scientists-just-detected-major-change-earths-oceans-linked-warming-climate-chris-mooney-the-washington-post/', 'https://science.feedback.org/review/california-dam-crisis-says-changing-climate-noah-diffenbaugh-new-york-times/', 'https://science.feedback.org/review/big-melt-global-sea-ice-record-low-usa-today-doyle-rice/', 'https://science.feedback.org/review/daily-mail-inflates-disagreement-scientists-data-handling-make-unsupported-accusation-data-manipulation/', 'https://science.feedback.org/review/wall-street-journal-fails-acknowledge-2016-warmest-year-record/', 'https://science.feedback.org/review/global-surface-temperatures-increasing-according-climate-projections-contrary-wall-street-journal-claim/', 'https://science.feedback.org/review/scientists-officially-declare-2016-hottest-year-record-chris-mooney-the-washington-post/', 'https://science.feedback.org/review/ocean-acidification-yet-another-wobbly-pillar-climate-alarmism-james-delingpole-the-spectator/', 'https://science.feedback.org/review/ocean-acidification-expected-harm-marine-ecosystems-overall-cherry-picking-can-lead-opposite-conclusion/', 'https://science.feedback.org/review/oceans-currently-acidifying-claims-contrary-contradict-observations/', 'https://science.feedback.org/review/arctic-ice-melt-already-affecting-weather-patterns-live-right-now-damian-carrington-the-guardian/', 'https://science.feedback.org/review/claim-arctic-warming-already-influencing-mid-latitude-weather-correct-still-investigation/', 'https://science.feedback.org/review/stunning-new-data-indicates-el-nino-drove-record-highs-global-temperatures-david-rose-daily-mail/', 'https://science.feedback.org/review/daily-mail-claim-2016-global-temperature-record-misleading/', 'https://science.feedback.org/review/great-barrier-reef-australia-coral-die-off-doyle-rice-usa-today/', 'https://science.feedback.org/review/higher-global-temperatures-sea-ice-record-lows-poles-brandon-miller-cnn/', 'https://science.feedback.org/review/the-phony-war-against-co2-the-wall-street-journal-rodney-nichols-harrison-schmitt/', 'https://science.feedback.org/review/wall-street-journal-op-ed-ignores-evidence-negative-impacts-increasing-co2/', 'https://science.feedback.org/review/sea-level-rise-accelerating-risk-associated-coastal-storm-surge-contrary-claim-forbes/', 'https://science.feedback.org/review/hillary-clinton-boards-climate-crisis-train-nowhere-roy-spencer-forbes/', 'https://science.feedback.org/review/sea-level-rise-those-non-disappearing-pacific-islands-bjorn-lomborg-wall-street-journal/', 'https://science.feedback.org/review/james-lovelock-interview-by-end-of-century-robots-will-have-taken-over-decca-aitkenhead-the-guardian/', 'https://science.feedback.org/review/climate-exaggeration-backfiring-robert-bradley-jr-forbes/', 'https://science.feedback.org/review/greenlands-huge-annual-ice-loss-is-even-worse-than-thought-damian-carrington-the-guardian/', 'https://science.feedback.org/review/flooding-of-coast-caused-by-global-warming-has-already-begun-justin-gillis-the-new-york-times/', 'https://science.feedback.org/review/sea-level-rise-much-1-2-meters-3-3-6-6-feet-year-2100/', 'https://science.feedback.org/review/arctic-will-be-ice-free-in-summer-next-year-robin-mckie-peter-wadhams-the-guardian/', 'https://science.feedback.org/review/louisiana-flooding-natural-disaster-weather-climate-change-oliver-milman-the-guardian/', 'https://science.feedback.org/review/environment-climate-change-records-broken-international-report-oliver-milman-the-guardian/', 'https://science.feedback.org/review/science-about-links-between-global-warming-and-massive-heat-waves-chris-mooney-washington-post/', 'https://science.feedback.org/review/heat-waves-u-s-already-become-frequent-intense-due-global-warming/', 'https://science.feedback.org/review/climate-change-emergency-jet-stream-shift-warning-global-warming-extreme-weather-gabriel-samuels-the-independent/', 'https://science.feedback.org/review/its-official-humans-are-making-the-earth-much-greener-washington-post-chris-mooney/', 'https://science.feedback.org/review/the-telegraph-bjorn-lomborg-in-many-ways-global-warming-will-be-good-thing/', 'https://science.feedback.org/review/great-barrier-reef-perish-2030s-andrew-freedman-mashable/', 'https://science.feedback.org/review/bjorn-lomborg-overheated-climate-alarm-wall-street-journal/', 'https://science.feedback.org/review/washington-post-chris-mooneys-earth-no-parallel-66-million-years/', 'https://science.feedback.org/review/scientists-are-exaggerating-carbon-threat-to-reefs-and-marine-life/', 'https://science.feedback.org/review/sea-level-rise-global-warming-climate-change/', 'https://science.feedback.org/review/patrick-michaels-the-climate-snow-job-the-wall-street-journal/', 'https://science.feedback.org/review/james-taylor-2015-was-not-even-close-to-hottest-year-on-record/', 'https://science.feedback.org/review/analysis-justin-gillis-2015-hottest-year-global-warming/', 'https://science.feedback.org/review/analysis-of-chris-mooney-scientists-say-human-greenhouse-gas-emissions-have-canceled-the-next-ice-age/', 'https://science.feedback.org/review/analysis-of-matt-ridley-benny-peiser-your-complete-guide-to-the-climate-debate/', 'https://science.feedback.org/review/analysis-of-john-kasich-climate-statements/', 'https://science.feedback.org/review/analysis-of-bernie-sanders-climate-statements/', 'https://science.feedback.org/review/analysis-of-marco-rubio-climate-statements/', 'https://science.feedback.org/review/analysis-of-hillary-clinton-climate-statements/', 'https://science.feedback.org/review/analysis-of-ted-cruz-climate-statements/', 'https://science.feedback.org/review/analysis-of-donald-trump-climate-statements/', 'https://science.feedback.org/review/analysis-of-martin-o-malley-climate-statements/', 'https://science.feedback.org/review/analysis-of-carly-fiorina-climate-statements/', 'https://science.feedback.org/review/analysis-of-chris-christie-climate-statements/', 'https://science.feedback.org/review/analysis-of-jeb-bush-climate-statements/', 'https://science.feedback.org/review/analysis-of-ben-carson-climate-statements/', 'https://science.feedback.org/review/analysis-of-rand-paul-climate-statements/', 'https://science.feedback.org/review/forbes-james-taylor-top-10-global-warming-lies-that-may-shock-you/', 'https://science.feedback.org/review/forbes-james-taylor-updated-nasa-data-polar-ice-not-receding-after-all/', 'https://science.feedback.org/review/andrew-freedman-mashable-hurricane-patricia-global-warming/', 'https://science.feedback.org/review/matthew-francis-forbes-even-without-drastic-predictions-coasts-are-in-danger/', 'https://science.feedback.org/review/andrew-freedman-south-carolina-flooding-climate-change/', 'https://science.feedback.org/review/betsy-mccaughey-wake-up-obama-climate-change-has-been-happening-forever/', 'https://science.feedback.org/review/tom-harris-deceptive-temperature-record-claims/', 'https://science.feedback.org/review/brad-plumer-el-nino-explained-why-this-years-could-be-one-of-the-strongest-on-record/', 'https://science.feedback.org/review/rolling-stone-eric-holthaus-point-of-no-return-climate-change-nightmares/', 'https://science.feedback.org/review/the-telegraph-christopher-booker-arctic-ice-has-made-fools-of-warmists/', 'https://science.feedback.org/review/the-telegraph-dan-hyde-earth-heading-for-mini-ice-age-within-15-years/', 'https://science.feedback.org/review/pope-francis-encyclical-laudato-si/', 'https://science.feedback.org/review/cnn-john-sutter-9-questions-on-rising-seas/', 'https://science.feedback.org/review/mashable-andrew-freedman-vanuatu-cyclone-pam-climate-change/', 'https://science.feedback.org/review/wall-street-journal-bjorn-lomborg-alarming-thing-climate-alarmism/']

In [ ]:
# @title (Code to find all verdicts–no need to run)
def getData(url, tag, c, title):
  labels = ["Accurate", "Correct", "Mostly accurate", "Mostly correct", "Lacks context", "Imprecise", "Partially correct", "Unsupported", "Misleading", "Inaccurate", "Incorrect", "Flawed reasoning"]
  html = requests.get(url)
  soup = BeautifulSoup(html.text, 'html.parser')
  info = soup.find_all(tag, class_=c)
  for i in info:
    i = i.get_text()
    if c == "":
      if i in labels: return i
    elif re.findall("\d", i): #If there is a number in i (meaning credibility score)
      return i
    else:
      return url

verdicts_list = []
for ind, url in enumerate(url_list):
  a = getData(url, "i", "", "Verdict")
  if a == None:
    b = getData(url, "span", "reviewed-content__score-num", "Credibility Score") # Doesn't return anything since this review is verdict label, not cred. score
    if b != None:
      b = b.strip("+")
    verdicts_list.append(b)
  else:
    verdicts_list.append(a)
print(verdicts_list)

In [41]:
# @title Initial verdicts (No need to run)
verdicts_list = ['Inaccurate', 'Inaccurate', 'Incorrect', 'Misleading', 'Incorrect', None, 'Inaccurate', 'Incorrect', 'Misleading', 'Misleading', 'Inaccurate', 'Incorrect', 'Inaccurate', 'Misleading', 'Misleading', 'Inaccurate', 'Incorrect', 'Incorrect', 'Inaccurate', 'Inaccurate', 'Incorrect', 'Incorrect', 'Incorrect', 'Incorrect', 'Inaccurate', 'Inaccurate', 'Inaccurate', 'Flawed reasoning', 'Inaccurate', 'Flawed reasoning', 'Incorrect', 'Incorrect', 'Incorrect', 'Incorrect', 'Incorrect', 'Incorrect', 'Incorrect', 'Incorrect', 'Mostly accurate', '-1.75', '-1.4', 'Misleading', '1.3', 'Inaccurate', None, 'Inaccurate', 'Incorrect', 'Inaccurate', 'Inaccurate', '-1.8', '-1.7', 'Inaccurate', '-1.8', 'Accurate', 'Inaccurate', 'Inaccurate', 'Incorrect', '1', '1.5', 'Inaccurate', 'Incorrect', '-2', 'Inaccurate', 'Incorrect', '-2', 'Inaccurate', 'Incorrect', 'Accurate', 'Misleading', '-1', 'Misleading', 'Imprecise', '-1', 'Misleading', 'Inaccurate', 'Lacks context', '-1', 'Correct', 'Incorrect', '-1', 'Accurate', 'Incorrect', '-1', '-2', 'Incorrect', 'Incorrect', '0.3', 'Flawed reasoning', 'Imprecise', 'Incorrect', 'Incorrect', 'Accurate', 'Mostly accurate', 'Mostly accurate', 'Inaccurate', 'Flawed reasoning', 'Flawed reasoning', 'Flawed reasoning', 'Incorrect', 'Inaccurate', None, 'Inaccurate', 'Misleading', 'Inaccurate', 'Inaccurate', 'Misleading', 'Incorrect', None, 'Inaccurate', 'Incorrect', 'Misleading', 'Inaccurate', 'Inaccurate', 'Incorrect', 'Inaccurate', '1.7', '-2', 'Inaccurate', '-0.8', 'Incorrect', 'Inaccurate', 'Misleading', 'Inaccurate', '-1.7', '-1.8', 'Incorrect', 'Incorrect', 'Inaccurate', 'Inaccurate', 'Incorrect', '-1.6', 'Incorrect', 'Misleading', 'Flawed reasoning', 'Incorrect', 'Incorrect', 'Inaccurate', 'Inaccurate', '1.3', 'Inaccurate', 'Incorrect', 'Incorrect', 'Incorrect', '1.7', 'Inaccurate', 'Inaccurate', None, '-1', '-1.4', '-2', 'Accurate', 'Flawed reasoning', 'Incorrect', 'Inaccurate', '-1.9', 'Flawed reasoning', 'Inaccurate', 'Misleading', '-2', 'Lacks context', '1.5', '0.2', '-0.3', '1.0', '1.5', '2', '2', '-2', '1.3', 'Incorrect', '1', '1.5', 'Inaccurate', '0.7', '-1.5', '1.3', 'Incorrect', 'Misleading', '-2', '1', 'Inaccurate', '1.7', '1.5', 'Incorrect', '0.8', None, '-1', '1.3', '1.8', '1', '-1.8', '-0.2', '-1.3', '-2', 'Inaccurate', '2', '-1.7', None, '1.5', None, 'Inaccurate', '0.3', '0.3', '1.7', '1.3', '-1.5', '0.7', '2', '-0.5', '-1.4', '-0.6', 'Incorrect', '0.8', '0.5', '-0.7', '-2', '0.7', 'Inaccurate', '1.4', 'Incorrect', '-1.7', '2', 'Incorrect', 'Mostly correct', '1.5', '1.6', 'Inaccurate', 'Inaccurate', '1', '-1.5', '1', '-0.7', '-1.8', 'Incorrect', '1.3', '1', 'Incorrect', '-2', 'Inaccurate', '-2', '-2', 'Misleading', 'Incorrect', 'Inaccurate', 'Flawed reasoning', '1.2', 'Incorrect', '0.3', 'Incorrect', '-2', '-2', 'Inaccurate', 'Incorrect', 'Misleading', 'Incorrect', 'Misleading', '2', '0.8', '1.7', '1.25', '1.5', '1.8', None, '-2', 'Mostly correct', 'Incorrect', '-1.5', '1.3', 'Mostly correct', '2', '1', '1.4', 'Unsupported', None, 'Misleading', '1.9', '-2', None, 'Inaccurate', '0.6', None, '-1.9', 'Misleading', '1.6', '2', '-2', None, 'Inaccurate', '-1.8', '-1.2', '-1.4', '-1.7', '1.6', '1.7', 'Correct', '-0.6', '1.3', '1.3', '1.7', 'Accurate', '-1.4', '0.6', None, '1.0', '-1.3', '1.5', '-2', '1.7', '-1.8', '-1.8', '1.9', '1.7', '-1.5', None, None, None, None, None, None, None, None, None, None, None, None, '-2', '-2', '1.4', '1', '1.3', '-2', '-1.9', '1.7', 'https://science.feedback.org/review/rolling-stone-eric-holthaus-point-of-no-return-climate-change-nightmares/', '-2', '-1.5', '0.9', None, '1.5', '-1.6']

In [ ]:
# @title (Code to find final urls and verdicts/cred scores–no need to run)
urls, verdicts = [], []
for ind, i in enumerate(verdicts_list):
  if i != None and i not in url_list:
    urls.append(url_list[ind])
    verdicts.append(i)
print(len(urls), len(verdicts))
print(urls)
print(verdicts)

In [ ]:
# @title (Code to find final sources–no need to run)
#Source
def getSource(sources, url):
  html = requests.get(url)
  soup = BeautifulSoup(html.text, 'html.parser')
  reference_ind = str(soup).index("Science Feedback is a non-partisan, non-profit organization dedicated to science education.")
  c = "line-height-lg"
  info = list(soup.find_all("p", class_=c))
  stopwords = ["source:", "published", "in"]
  ret_val = ""
  if len(info) == 3:
    ret_val = info[1].get_text()
  elif len(info) == 2:
    ret_val = info[0].get_text()
  else:
    c = "reviewed-content__figcaption line-height-md flex gap-xs"
    info = list(soup.find_all("figcaption", class_=c))
    ret_val = info[0].get_text()
    source_ind = str(soup).index(str(info[0]))
    if source_ind > reference_ind: # If the source is below the text "Support our work," and there was no source for "line-height-lg," then that means the source is for one of the recommended reviews at the bottom of the page
      return url
  ret_val = ret_val.split()
  resultwords  = [word for ind, word in enumerate(ret_val) if ind > 1 or word.lower() not in stopwords]
  result = ' '.join(resultwords)
  if result:
    return result
  else: return url
sources = []
for ind, url in enumerate(urls):
  res = getSource(sources, url)
  print(ind, res)
  sources.append(res)
print("TOTAL Num Of SOURCES & SET of sources: ", len(sources), len(set(sources))) #set(sources) doesn't matter–sometimes the same article is reviewed multiple times

In [45]:
# @title **Final urls, verdicts/cred scores, and sources (RUN)**
urls = ['https://science.feedback.org/review/pyramid-shaped-peaks-antarctica-naturally-carved-by-glaciers-not-constructed-by-ancient-civilization/', 'https://science.feedback.org/review/sea-levels-have-risen-for-over-100-years-despite-misleading-photos-social-media/', 'https://science.feedback.org/review/colorful-auroras-seen-may-2024-caused-by-mass-ejections-from-sun-not-haarp-experiments/', 'https://science.feedback.org/review/no-evidence-significant-influence-volcanoes-solar-variability-on-recent-climate-change-contrary-judith-curry-claims-prageru-video/', 'https://science.feedback.org/review/carbon-isotopes-do-not-show-humans-climate-impacts-too-small-notice-despite-the-daily-sceptic-inaccurate-claim/', 'https://science.feedback.org/review/data-shows-temperatures-rising-greenland-world-current-global-warming-driven-co2-not-solar-activity/', 'https://science.feedback.org/review/arctic-sea-ice-melting-over-past-decades-with-projections-summer-ice-free-conditions-mid-century/', 'https://science.feedback.org/review/optimal-atmospheric-co2-dinosaurs-plants-harmful-humans-current-concentration-higher-homo-sapiens-ever-experienced/', 'https://science.feedback.org/review/study-calls-people-grow-their-own-food-years-come-despite-finding-higher-carbon-footprint-compared-conventional-agriculture/', 'https://science.feedback.org/review/us-proposed-bills-hearings-dont-confirm-chemtrails-exist-not-geoengineering-strategy/', 'https://science.feedback.org/review/evidence-greenhouse-gasses-cause-global-warming-denied-willie-soon-tucker-carlson-interview-mass-social-media-climate-misinformation/', 'https://science.feedback.org/review/eu-not-passing-law-seize-scrap-cars-under-climate-agenda-contrary-peter-sweden-claim/', 'https://science.feedback.org/review/contrary-widespread-misrepresentation-new-study-finds-extremely-minimal-impact-human-breathing-climate/', 'https://science.feedback.org/review/claim-cow-burps-not-important-contributor-climate-change-ignores-full-range-greenhouse-gas-emissions-beef-production/', 'https://science.feedback.org/review/uncertainties-about-future-climate-change-1970s-does-not-invalidate-today-evidence-reality-global-warming/', 'https://science.feedback.org/review/water-vapor-greenhouse-gas-not-major-driver-global-warming/', 'https://science.feedback.org/review/video-ian-plimer-incorrectly-states-human-co2-emissions-not-responsible-for-increased-atmospheric-co2-concentrations-global-warming/', 'https://science.feedback.org/review/global-warming-matches-model-predictions-well-misconstrued-claims-new-photosynthesis-study/', 'https://science.feedback.org/review/wall-street-journal-questions-decades-scientific-evidence-demonstrating-elevated-atmospheric-co2-causes-global-warming/', 'https://science.feedback.org/review/claim-current-climate-change-explained-natural-cycles-volcanic-activity-does-not-have-scientific-support/', 'https://science.feedback.org/review/clintel-group-inaccurately-represents-climate-science-declaration-no-climate-emergency-once-again/', 'https://science.feedback.org/review/three-decades-chemtrail-haarp-geoengineering-conspiracy-theory-remains-popular-despite-lack-evidence-no-scientific-basis/', 'https://science.feedback.org/review/magnetic-poles-reversals-not-cataclysmic-events-adam-and-eve-theory-jimmy-corsetti-joe-rogan-podcast/', 'https://science.feedback.org/review/headline-temperatures-on-earth-are-increasing-and-the-rise-is-drastically-outpacing-previous-natural-changes-in-the-planets-climate/', 'https://science.feedback.org/review/antarctic-ice-coverage-not-increasing-record-levels-nor-continent-getting-colder-contrary-to-curry-easterbrook-cfact-movie/', 'https://science.feedback.org/review/strands-cloud-behind-aeroplanes-trails-condensed-water-contrails-not-chemtrails-some-believe/', 'https://science.feedback.org/review/pre-human-changes-climate-not-negate-contemporary-human-influence-modern-warming/', 'https://science.feedback.org/review/link-between-co2-earth-temperature-well-established-despite-claims-fox-news-tom-harris/', 'https://science.feedback.org/review/scientists-found-modern-climate-change-caused-entirely-by-human-activity-contrary-recent-speculation-joe-rogan-experience/', 'https://science.feedback.org/review/the-epoch-times-prints-range-of-inaccurate-misleading-claims-climate-changes-impacts-causes-patrick-moore/', 'https://science.feedback.org/review/scientific-evidence-indicates-modern-warming-driven-by-co2-contrary-to-richard-lindzen/', 'https://science.feedback.org/review/letter-there-is-no-climate-emergency-repeats-inaccurate-claims-about-climate-science-daily-sceptic-toby-young/', 'https://science.feedback.org/review/carbon-dioxide-emissions-humans-major-driver-climate-change-jerome-corsi-claims-cbn-news/', 'https://science.feedback.org/review/the-sun-isnt-responsible-for-current-climate-change-contrary-claims-suspicious0bservers-ben-davidson/', 'https://science.feedback.org/review/warming-earths-surface-oceans-continues-apace-contrary-to-claims-daily-sceptic-chris-morrison/', 'https://science.feedback.org/review/earths-surface-warm-due-greenhouse-gas-christopher-monckton-watts-up-with-that/', 'https://science.feedback.org/review/robust-scientific-evidence-supports-human-activity-drives-global-warming-contrary-to-claims-co2-coalition-blog-post-andy-may/', 'https://science.feedback.org/review/more-than-75-percent-amazon-rainforest-losing-resilience-the-washington-post-sarah-kaplan/', 'https://science.feedback.org/review/wall-street-journal-steven-koonin-publishes-misleading-claims-climate-change-influences-greenland-ice-melt/', 'https://science.feedback.org/review/prageru-video-on-climate-change-repeats-a-range-of-misleading-claims-by-steven-koonin/', 'https://science.feedback.org/review/coral-cover-great-barrier-reef-improved-2021-but-doesnt-mean-reef-growing-quickly-contrary-to-daily-wire/', 'https://science.feedback.org/review/rain-fell-on-greenland-summit-for-the-first-time-on-record-as-accurately-reported-by-cnn/', 'https://science.feedback.org/review/a-2014-study-showed-that-the-us-corn-belt-is-one-of-the-biggest-primary-producers-on-earth-in-july-but-didnt-show-that-it-produces-more-oxygen-than-the-amazon/', 'https://science.feedback.org/review/sea-temperatures-around-the-great-barrier-reef-have-increased-over-the-last-150-years-contrary-to-claim-at-wattsupwiththat/', 'https://science.feedback.org/review/solar-forcing-is-not-the-main-cause-of-current-global-warming-contrary-to-claim-by-alex-newman-in-the-epoch-times/', 'https://science.feedback.org/review/despite-2021-being-a-good-year-for-coral-health-coral-in-the-great-barrier-reef-has-declined-over-the-past-decade-and-is-threatened-by-climate-change-contrary-to-claims-by-peter-ridd/', 'https://science.feedback.org/review/youtube-video-falsely-claims-that-climate-models-systematically-over-estimated-warming-and-that-we-are-heading-towards-an-ice-age/', 'https://science.feedback.org/review/gregory-wrightstone-article-in-the-washington-times-presents-list-of-false-and-misleading-statements-about-the-impacts-of-co2-and-climate-change-co2-coalition/', 'https://science.feedback.org/review/wall-street-journal-article-repeats-multiple-incorrect-and-misleading-claims-made-in-steven-koonins-new-book-unsettled-steven-koonin/', 'https://science.feedback.org/review/bidens-climate-plan-doesnt-say-anything-about-limiting-meat-consumption-contrary-to-daily-mail-fox-news-claim/', 'https://science.feedback.org/review/video-promoted-by-john-stossel-for-earth-day-relies-on-incorrect-and-misleading-claims-about-climate-change/', 'https://science.feedback.org/review/a-third-of-antarctic-ice-shelf-risks-collapse-at-4c-above-pre-industrial-levels-as-accurately-described-in-cnn-article/', 'https://science.feedback.org/review/in-viral-turning-point-usa-video-candace-owens-and-charlie-kirk-falsely-claim-there-is-no-evidence-of-global-warming-and-scientists-dont-know-the-cause/', 'https://science.feedback.org/review/deserts-cool-the-planet-by-reflecting-solar-radiation-to-space-vegetated-areas-have-an-overall-warming-effect-so-planting-trees-in-deserts-doesnt-necessarily-cool-the-planet-the-guardian/', 'https://science.feedback.org/review/solar-geoengineering-isnt-happening-or-damaging-the-planet-aircraft-contrails-are-formed-by-water-vapor-not-chemicals/', 'https://science.feedback.org/review/destruction-of-the-amazon-forest-very-likely-contributes-to-global-warming-as-accurately-described-in-the-national-geographic-article/', 'https://science.feedback.org/review/atlantic-ocean-circulation-system-is-slowing-down-as-accurately-described-in-the-new-york-times-article/', 'https://science.feedback.org/review/scientists-didnt-announce-impending-environmental-catastrophes-every-decade-since-the-1970s/', 'https://science.feedback.org/review/the-scientific-research-facility-haarp-cannot-create-natural-disasters-or-influence-human-thoughts-and-movements-contrary-to-online-claims/', 'https://science.feedback.org/review/breitbart-article-makes-numerous-false-claims-about-the-impacts-of-climate-change-based-on-global-warming-policy-foundation-post-delingpole-goklany/', 'https://science.feedback.org/review/noaa-shows-clear-global-warming-trends-over-the-past-58-years-based-on-radiosonde-data/', 'https://science.feedback.org/review/prageru-post-by-william-happer-uses-flawed-reasoning-to-claim-that-climate-models-always-fail/', 'https://science.feedback.org/review/video-interview-of-ian-plimer-at-sky-news-falsely-claims-that-a-new-study-announces-an-incoming-ice-age-partly-based-on-an-incorrect-daily-mail-headline/', 'https://science.feedback.org/review/rates-of-global-sea-level-rise-have-accelerated-since-1900-contrary-to-bloggers-claims/', 'https://science.feedback.org/review/scientific-studies-established-clear-links-between-human-caused-increased-in-atmospheric-co2-and-global-warming-patrick-moore/', 'https://science.feedback.org/review/the-clothing-industry-produces-3-to-10-of-global-greenhouse-gas-emissions-as-accurately-claimed-in-patagonia-post/', 'https://science.feedback.org/review/brush-rake-attachments-are-primarily-used-to-pile-up-heavy-forest-debris-as-part-of-active-thinning-harvesting-and-removal-projects/', 'https://science.feedback.org/review/guardian-article-on-arctic-methane-emissions-lacks-important-context-jonathan-watts/', 'https://science.feedback.org/review/sea-ice-loss-due-to-climate-change-is-the-biggest-threat-to-polar-bear-survival-and-has-already-led-to-declines-in-some-polar-bear-subpopulations-contrary-to-climate-realism-video-m/', 'https://science.feedback.org/review/climate-change-can-make-it-harder-for-the-amazon-rainforest-to-grow-back-from-deforestation-but-that-does-not-mean-40-of-it-will-now-turn-into-a-savanna/', 'https://science.feedback.org/review/article-by-the-daily-caller-oversimplifies-drivers-of-wildfires-and-downplays-role-of-climate-change-chris-white/', 'https://science.feedback.org/review/climate-change-forest-management-and-several-other-causes-contribute-to-wildfire-severity-and-total-area-burned-in-the-western-united-states/', 'https://science.feedback.org/review/western-us-wildfires-are-not-the-result-of-widespread-arson/', 'https://science.feedback.org/review/planting-trees-can-help-mitigate-some-aspects-of-climate-change-but-it-cannot-solve-all-environmental-crises/', 'https://science.feedback.org/review/article-by-cnn-exaggerates-studys-implications-for-future-greenland-ice-loss/', 'https://science.feedback.org/review/the-long-term-survival-of-polar-bears-is-threatened-by-loss-of-sea-ice-due-to-global-warming-new-study-confirms-henry-fountain/', 'https://science.feedback.org/review/greenhouse-gases-cause-global-warming-by-trapping-infra-red-radiations-not-by-causing-more-holes-in-the-ozone-layer/', 'https://science.feedback.org/review/article-by-michael-shellenberger-mixes-accurate-and-inaccurate-claims-in-support-of-a-misleading-and-overly-simplistic-argumentation-about-climate-change/', 'https://science.feedback.org/review/study-evaluates-natural-and-human-causes-of-recent-rapid-warming-rate-at-the-south-pole/', 'https://science.feedback.org/review/climate-scientists-agree-that-human-caused-greenhouse-gas-emissions-are-primarily-responsible-for-climate-change-contrary-to-claims-in-clear-energy-alliance-video/', 'https://science.feedback.org/review/article-in-the-guardian-misleads-readers-about-sensitivity-of-climate-models-by-narrowly-focusing-on-single-study-jonathan-watts/', 'https://science.feedback.org/review/video-from-prageru-makes-several-incorrect-and-misleading-claims-about-climate-change-richard-lindzen/', 'https://science.feedback.org/review/low-solar-activity-has-little-effect-on-earths-climate-contrary-to-claim-in-the-sun/', 'https://science.feedback.org/review/the-global-polar-bear-population-is-threatened-by-loss-of-sea-ice-contrary-to-pragerus-video-claim/', 'https://science.feedback.org/review/article-in-business-insider-accurately-describes-results-from-a-study-estimating-up-to-3-billion-people-could-live-in-much-warmer-temperatures-by-2070/', 'https://science.feedback.org/review/diesel-cars-are-a-major-source-of-no2-emissions-in-european-cities-contrary-to-online-claim/', 'https://science.feedback.org/review/claim-that-co2-is-cooling-the-planet-is-misleading-anthropogenic-co2-emissions-overall-cause-global-warming/', 'https://science.feedback.org/review/human-activities-have-dramatically-increased-atmospheric-co2-levels-causing-imbalances-in-the-global-carbon-cycle/', 'https://science.feedback.org/review/human-induced-increases-in-greenhouse-gases-are-the-primary-driver-of-global-warming-contrary-to-claims-in-cfact-article/', 'https://science.feedback.org/review/scientists-observe-most-widespread-bleaching-event-ever-recorded-at-the-great-barrier-reef/', 'https://science.feedback.org/review/the-antarctic-ozone-layer-is-recovering-but-the-independent-claim-fails-to-grasp-significance-of-study-it-relies-on/', 'https://science.feedback.org/review/deforestation-can-facilitate-the-emergence-and-spread-of-some-infectious-diseases/', 'https://science.feedback.org/review/global-data-contradict-claim-of-no-acceleration-in-sea-level-rise/', 'https://science.feedback.org/review/global-warming-and-the-covid-19-pandemic-are-unrelated-issues-one-will-not-protect-us-from-the-other-life-site-news/', 'https://science.feedback.org/review/temperature-trends-in-the-u-s-are-consistent-with-warming-around-the-world-contrary-to-electroverse-claim-cap-allon/', 'https://science.feedback.org/review/co2-can-increase-plant-growth-in-greenhouses-while-also-negatively-affecting-ecosystems-and-human-societies-mike-adams-natural-news/', 'https://science.feedback.org/review/electroverse-article-incorrectly-claims-the-sun-is-behind-climate-change/', 'https://science.feedback.org/review/sea-levels-rose-faster-in-the-past-century-than-in-previous-time-periods/', 'https://science.feedback.org/review/data-from-glaciers-around-the-world-show-most-are-shrinking-as-a-consequence-of-global-warming/', 'https://science.feedback.org/review/current-climate-warming-is-rapid-and-occurring-on-a-global-scale-unlike-past-periods-of-regional-climate-fluctuations/', 'https://science.feedback.org/review/sea-levels-have-risen-throughout-the-20th-century-contrary-to-claim-in-online-article/', 'https://science.feedback.org/review/noaa-does-not-fraudulently-adjust-data-to-create-the-illusion-of-a-global-warming-trend-investors/', 'https://science.feedback.org/review/co2-emissions-from-human-activities-have-imbalanced-the-atmospheric-carbon-budget-significantly-contributing-to-climate-change-contrary-to-online-claim-issues-insights/', 'https://science.feedback.org/review/claims-of-a-coming-30-year-mini-ice-age-are-not-supported-by-science-the-sun/', 'https://science.feedback.org/review/simple-measurements-demonstrate-that-co2-is-a-greenhouse-gas-tim-ball/', 'https://science.feedback.org/review/facebook-meme-incorrectly-claims-past-climate-variations-contradict-human-influence-on-current-climate/', 'https://science.feedback.org/review/climate-change-bushfires-australia-breitbart-newspunch/', 'https://science.feedback.org/review/volcanic-co2-emissions-are-known-to-be-much-smaller-than-human-caused-emissions-james-edward-kamis/', 'https://science.feedback.org/review/article-falsely-attributes-invented-quotes-to-james-hansen-dmitry-baxter/', 'https://science.feedback.org/review/ice-cube-meme-misrepresents-physics-of-sea-level-rise-to-claim-melting-ice-has-no-effect/', 'https://science.feedback.org/review/1977-coming-ice-age-time-magazine-cover-is-a-fake/', 'https://science.feedback.org/review/abc-article-effectively-illustrates-important-climate-trends-for-australian-readers-tim-leslie-joshua-byrd-nathan-hoad/', 'https://science.feedback.org/review/ian-plimer-op-ed-in-the-australian-again-presents-long-list-of-false-claims-about-climate/', 'https://science.feedback.org/review/bedrock-heat-flow-studies-made-no-claim-about-human-caused-melting-climate-change-dispatch-james-edward-kamis/', 'https://science.feedback.org/review/new-york-times-op-ed-claiming-scientists-underestimated-climate-change-lacks-supporting-evidence-eugene-linden/', 'https://science.feedback.org/review/claim-of-a-coming-ice-age-misrepresents-the-study-it-relies-on/', 'https://science.feedback.org/review/human-activities-release-far-more-carbon-dioxide-than-volcanoes-do/', 'https://science.feedback.org/review/letter-to-un-was-not-signed-by-500-experts-on-climate-science-breitbart/', 'https://science.feedback.org/review/photo-meme-of-sydney-harbour-incorrectly-claims-no-sea-level-rise-has-occurred/', 'https://science.feedback.org/review/telegraph-article-misleads-with-false-balance-mixing-in-unsupported-and-inaccurate-claims-sarah-knapton/', 'https://science.feedback.org/review/letter-signed-by-500-scientists-relies-on-inaccurate-claims-about-climate-science/', 'https://science.feedback.org/review/2c-not-known-point-of-no-return-as-jonathan-franzen-claims-new-yorker/', 'https://science.feedback.org/review/there-is-overwhelming-evidence-that-climate-change-is-human-caused-townhall/', 'https://science.feedback.org/review/claim-that-the-earth-has-cooled-since-medieval-or-roman-times-is-false-marc-morano/', 'https://science.feedback.org/review/claim-of-no-us-warming-since-2005-is-directly-contradicted-by-the-data-it-is-based-on/', 'https://science.feedback.org/review/earths-orbit-cannot-explain-modern-climate-change/', 'https://science.feedback.org/review/washington-examiner-op-ed-cherry-picks-data-to-mislead-readers-about-climate-models-patrick-michaels-caleb-stewart-rossiter/', 'https://science.feedback.org/review/prediction-extinction-rebellion-climate-change-will-kill-6-billion-people-unsupported-roger-hallam-bbc/', 'https://science.feedback.org/review/rick-perrys-claim-that-us-is-leading-the-world-in-emission-cuts-is-misleading-fox-news/', 'https://science.feedback.org/review/its-not-true-that-co2-only-increases-after-temperature-rises-william-happer-ice-age-now/', 'https://science.feedback.org/review/claim-that-cosmic-rays-are-a-crucial-player-for-current-climate-change-is-unsupported-nir-shaviv-electroverse-global-warming-policy-foundation/', 'https://science.feedback.org/review/yet-another-express-article-falsely-warns-of-a-solar-induced-ice-age-sean-martin/', 'https://science.feedback.org/review/research-does-not-show-a-medieval-warm-period-warmer-than-the-present-day/', 'https://science.feedback.org/review/hosted-by-former-australian-senator-tony-heller-repeats-false-claim-that-scientists-fake-the-warming-trend/', 'https://science.feedback.org/review/nbc-news-story-accurately-covers-research-on-two-millennia-of-climate-history-jaclyn-jeffrey-wilensky/', 'https://science.feedback.org/review/the-earth-was-not-warmer-in-medieval-times-town-hall-gregory-rummo/', 'https://science.feedback.org/review/sky-news-australia-interview-falsely-claims-that-global-cooling-is-coming-soon/', 'https://science.feedback.org/review/non-peer-reviewed-manuscript-falsely-claims-natural-cloud-changes-can-explain-global-warming/', 'https://science.feedback.org/review/on-fox-news-patrick-michaels-falsely-claims-humans-are-only-responsible-for-half-of-global-warming/', 'https://science.feedback.org/review/popular-article-in-the-independent-accurately-summarizes-june-heat-in-europe-conrad-duncan/', 'https://science.feedback.org/review/nasa-did-not-create-global-warming-by-manipulating-data-tony-heller-steven-goddard/', 'https://science.feedback.org/review/claim-that-electric-vehicles-cause-more-carbon-emissions-than-diesel-misrepresents-reality/', 'https://science.feedback.org/review/financial-post-commentary-misrepresents-scientific-understanding-of-weather-extremes-ross-mckitrick/', 'https://science.feedback.org/review/iflscience-story-on-speculative-report-provides-little-scientific-context-james-felton/', 'https://science.feedback.org/review/the-australian-commentary-by-ian-plimer-relies-on-false-claims-to-make-its-case/', 'https://science.feedback.org/review/its-true-that-the-current-carbon-dioxide-level-is-higher-than-any-time-in-human-existence/', 'https://science.feedback.org/review/video-falsely-claiming-to-be-barron-trump-also-falsely-claims-humans-arent-causing-climate-change/', 'https://science.feedback.org/review/there-are-clear-evidence-for-climate-change-contrary-to-claims-by-john-coleman/', 'https://science.feedback.org/review/claim-in-the-express-that-low-solar-activity-is-bringing-cold-weather-is-false/', 'https://science.feedback.org/review/breitbart-article-baselessly-claims-a-study-of-past-climate-invalidates-human-caused-climate-change-john-nolte/', 'https://science.feedback.org/review/blaze-article-badly-misrepresents-study-of-greenlands-jakobshavn-glacier/', 'https://science.feedback.org/review/caleb-rossiter-falsely-claims-that-climate-models-are-running-very-hot/', 'https://science.feedback.org/review/senator-sanders-claim-that-climate-change-is-making-tornadoes-worse-isnt-supported-by-published-research/', 'https://science.feedback.org/review/western-journal-op-ed-deceives-readers-with-completely-unsupported-claims-jay-lehr-tom-harris/', 'https://science.feedback.org/review/claim-about-climate-impacts-of-cows-vs-cars-needs-a-little-explanation-financial-times/', 'https://science.feedback.org/review/new-york-times-coverage-of-ipcc-report-clearly-presents-conclusions-coral-davenport/', 'https://science.feedback.org/review/guardian-story-on-climate-impacts-of-diet-gets-mixed-reviews-from-scientists-damian-carrington/', 'https://science.feedback.org/review/story-on-congressmans-incorrect-claims-about-sea-level-rise-could-have-corrected-them-more-explicitly/', 'https://science.feedback.org/review/bbc-article-on-ipcc-report-is-mostly-accurate-but-could-use-some-clarification/', 'https://science.feedback.org/review/cnn-accurately-covers-latest-ipcc-report-brandon-miller/', 'https://science.feedback.org/review/usa-today-story-updates-readers-on-trend-in-monthly-global-temperatures-doyle-rice/', 'https://science.feedback.org/review/washington-post-article-accurately-discusses-warm-arctic-weather-event-jason-samenow/', 'https://science.feedback.org/review/friends-of-science-video-promoted-by-youtube-presents-long-list-of-climate-myths-steve-goreham/', 'https://science.feedback.org/review/popular-story-fox2news-accurately-describes-2017-global-temperature-sea-ice-extent-cnn/', 'https://science.feedback.org/review/metros-claims-of-coming-mini-ice-age-have-no-basis-in-reality/', 'https://science.feedback.org/review/washington-post-accurately-describes-ocean-warming-study-with-potential-implications-for-future-carbon-budget-chris-mooney-brady-dennis/', 'https://science.feedback.org/review/guardian-story-accurately-describes-study-on-environmental-impacts-of-our-food-system-damian-carrington/', 'https://science.feedback.org/review/on-bbc-newsnight-myron-ebell-falsely-claims-climate-models-exaggerate-warming/', 'https://science.feedback.org/review/guardian-conveys-statement-scientists-stressing-role-of-deforestation-climate-change-oliver-milman/', 'https://science.feedback.org/review/usa-today-op-ed-ignores-evidence-to-claim-climate-change-had-no-role-in-hurricane-florence-roy-spencer/', 'https://science.feedback.org/review/guardian-story-accurately-covers-sea-ice-event-but-makes-unsupported-connection-weather-patterns-gulf-stream-jonathan-watts/', 'https://science.feedback.org/review/ian-plimer-wrongly-claims-that-carbon-dioxide-emissions-do-not-cause-climate-change/', 'https://science.feedback.org/review/ian-plimers-op-ed-incorrectly-claims-human-caused-emissions-are-short-lived/', 'https://science.feedback.org/review/op-ed-in-the-australian-gets-nearly-every-fact-wrong-ian-plimer/', 'https://science.feedback.org/review/guardian-potential-future-hothouse-climate-generally-accurate-but-misstates-details-jonathan-watts/', 'https://science.feedback.org/review/president-trumps-claim-that-water-supply-policy-has-worsened-california-wildfires-is-baseless/', 'https://science.feedback.org/review/washington-post-story-puts-recent-weather-extremes-in-accurate-climate-change-context-joel-achenbach-angela-fritz/', 'https://science.feedback.org/review/npr-story-accurately-describes-ecological-consequences-of-altered-spring-timings-in-a-warming-climate-nathan-rott/', 'https://science.feedback.org/review/fred-singer-incorrectly-claims-sea-level-rise-is-not-caused-by-climate-change/', 'https://science.feedback.org/review/usa-today-paleoclimate-study-mostly-accurate-lacks-clarity-doyle-rice/', 'https://science.feedback.org/review/financial-post-commentary-misleads-warming-effect-greenhouse-gas-emissions-cherry-picking-studies-ross-mckitrick/', 'https://science.feedback.org/review/washington-post-article-accurately-describes-latest-estimate-accelerating-antarctic-ice-loss-chris-mooney/', 'https://science.feedback.org/review/national-geographic-hurricanes-moving-slower-global-warming-craig-welch/', 'https://science.feedback.org/review/new-york-times-story-accurately-describes-rio-grandes-climate-context-henry-fountain/', 'https://science.feedback.org/review/wall-street-journal-commentary-grossly-misleads-readers-about-science-of-sea-level-rise-fred-singer/', 'https://science.feedback.org/review/business-insider-highlights-health-impacts-climate-change-aspects-are-misleading-kevin-loria/', 'https://science.feedback.org/review/the-australian-coverage-great-barrier-reef-perception-scientists-divided-global-warming-graham-lloyd/', 'https://science.feedback.org/review/investors-business-daily-editorial-misrepresents-study-to-claim-plants-will-prevent-dangerous-climate-change/', 'https://science.feedback.org/review/rush-limbaugh-falsely-claims-there-is-no-evidence-of-human-caused-global-warming/', 'https://science.feedback.org/review/washington-post-accurately-covers-permafrost-study-albeit-under-a-somewhat-sensational-headline-chris-mooney/', 'https://science.feedback.org/review/financial-post-publishes-misleading-opinion-misrepresents-science-polar-bears-peril-susan-crockford/', 'https://science.feedback.org/review/satellite-measurements-accelerating-sea-level-rise-cnn-accurately-reports-brandon-miller/', 'https://science.feedback.org/review/president-trumps-claim-growing-ice-not-reflect-reality/', 'https://science.feedback.org/review/national-geographic-video-starving-polar-bear-should-have-clarified-uncertain-link-climate-change-sarah-gibbens/', 'https://science.feedback.org/review/atlantic-climate-implications-reduced-beef-consumption-provide-clearer-context-james-hamblin/', 'https://science.feedback.org/review/new-york-times-iceberg-larsen-c-size-delaware-jugal-patel-justin-gillis/', 'https://science.feedback.org/review/new-york-times-accurately-covers-2017-record-low-arctic-winter-sea-ice-extent-henry-fountain/', 'https://science.feedback.org/review/conservative-tribune-post-falsely-claims-cancelled-arctic-research-cruise-is-evidence-against-climate-change-benjamin-arie/', 'https://science.feedback.org/review/iflscience-story-florida-sea-level-rise-somewhat-unclear-generally-correct/', 'https://science.feedback.org/review/new-york-times-news-coverage-2016-global-temperature-data-accurate-summary-justin-gillis/', 'https://science.feedback.org/review/futurism-story-great-barrier-reef-compromised-sensational-headline/', 'https://science.feedback.org/review/breitbart-california-fires-misleads-omitting-explanation-climates-influence-joel-pollak/', 'https://science.feedback.org/review/daily-caller-uncritically-reports-misleading-satellite-temperature-study-michael-bastasch/', 'https://science.feedback.org/review/prof-john-christy-incorrectly-claims-to-show-climate-models-are-too-sensitive-to-carbon-dioxide/', 'https://science.feedback.org/review/antarctica-doomsday-glaciers-could-flood-coastal-cities-grist-eric-holthaus/', 'https://science.feedback.org/review/guardian-sea-level-rise-cities-fails-make-timescale-clear/', 'https://science.feedback.org/review/independent-makes-giant-leap-global-warming-worse-than-thought-single-study-andrew-griffin/', 'https://science.feedback.org/review/commentary-australian-ignores-evidence-misrepresents-research-falsely-claiming-humans-not-responsible-climate-change-ian-plimer/', 'https://science.feedback.org/review/bbc-article-ocean-acidification-accurate-brief-roger-harrabin/', 'https://science.feedback.org/review/former-prime-minister-tony-abbott-wrongly-claims-australia-warmed-far-less-data-show/', 'https://science.feedback.org/review/new-york-times-straightforward-answers-common-climate-questions-accurate-justin-gillis/', 'https://science.feedback.org/review/daily-wire-article-incorrectly-claims-climate-models-have-overestimated-warming/', 'https://science.feedback.org/review/daily-wire-article-misunderstands-study-carbon-budget-along-fox-news-telegraph-daily-mail-breitbart-james-barrett/', 'https://science.feedback.org/review/global-warming-fire-suppression-practices-boost-wildfires-us-west-correctly-reported-the-atlantic-robinson-meyer/', 'https://science.feedback.org/review/incorrect-claim-global-warming-mostly-natural-based-on-study-that-cant-support-conclusion-jennifer-marohasy-john-abbot/', 'https://science.feedback.org/review/climate-change-likely-helped-increase-hurricane-harveys-rainfall-early-say-exactly-much/', 'https://science.feedback.org/review/atlantic-accurately-explores-climate-context-tropical-storm-harvey-robinson-meyer/', 'https://science.feedback.org/review/new-york-times-accurately-assesses-the-state-of-alaskan-permafrost-henry-fountain/', 'https://science.feedback.org/review/extreme-weather-events-clearly-becoming-common-contrast-lord-lawsons-claim/', 'https://science.feedback.org/review/lord-lawson-falsely-claims-global-temperatures-declined-last-decade/', 'https://science.feedback.org/review/forbes-article-accurately-describes-research-atlantic-ocean-circulation-weakening-headline-goes-farther-trevor-nace/', 'https://science.feedback.org/review/wall-street-journal-op-ed-economic-consequences-climate-change-found-naive-scientists-david-henderson-john-cochrane/', 'https://science.feedback.org/review/2017-track-among-hottest-year-recorded-scientists-not-surprised-thinkprogress-article-suggests-joe-romm/', 'https://science.feedback.org/review/scientists-explain-what-new-york-magazine-article-on-the-uninhabitable-earth-gets-wrong-david-wallace-wells/', 'https://science.feedback.org/review/breitbart-article-falsely-claims-measured-global-warming-fabricated-james-delingpole/', 'https://science.feedback.org/review/earth-is-not-at-risk-of-becoming-a-hothouse-like-venus-as-stephen-hawking-claimed-bbc/', 'https://science.feedback.org/review/the-atlantic-accurately-reports-study-economic-impacts-continued-climate-change-us-robinson-meyer/', 'https://science.feedback.org/review/new-york-times-story-highlights-the-growing-number-of-extremely-hot-days-in-a-warming-world-brad-plumer-n-popovich/', 'https://science.feedback.org/review/energy-secretary-rick-perry-incorrectly-claims-co2-not-primary-cause-climate-change/', 'https://science.feedback.org/review/breitbart-misrepresents-research-58-scientific-papers-falsely-claim-disprove-human-caused-global-warming-james-delingpole/', 'https://science.feedback.org/review/breitbart-falsely-claims-58-studies-refute-human-caused-global-warming/', 'https://science.feedback.org/review/playing-semantics-misleading-breitbart-article-downplays-us-contribution-to-climate-change/', 'https://science.feedback.org/review/paris-agreement-op-ed-us-senator-ted-cruz-misrepresents-costs-benefits-reducing-greenhouse-gas-emissions/', 'https://science.feedback.org/review/natural-variability-can-not-explain-modern-global-warming-heartland-institute-report-claims/', 'https://science.feedback.org/review/lull-solar-activity-little-effect-global-temperatures-claims-global-cooling-not-based-science/', 'https://science.feedback.org/review/global-warming-not-ceased-heartland-institute-report-falsely-claims/', 'https://science.feedback.org/review/co2-greenhouse-gas-caused-warming-past-climate-changes-notably-feedback-amplifying-factors/', 'https://science.feedback.org/review/new-york-times-series-accurately-describes-research-antarctic-ice-sheets-sea-level-rise-justin-gillis/', 'https://science.feedback.org/review/heartland-institute-reports-claim-climate-models-sensitive-co2-not-reflect-evidence/', 'https://science.feedback.org/review/alaskas-vicious-cycle-warming-tundra-spews-co2-speeding-up-warming-joe-romm-think-progress/', 'https://science.feedback.org/review/heartland-institute-report-incorrectly-claims-no-evidence-human-impacts-melting-ice/', 'https://science.feedback.org/review/global-quackery-earth-not-warmed-past-19-years-new-study-finds-joseph-curl-the-daily-wire/', 'https://science.feedback.org/review/another-arctic-ice-panic-world-temperatures-plummet-the-telegraph-christopher-booker/', 'https://science.feedback.org/review/global-sea-level-rise-accelerating-despite-heartland-institute-reports-claims-otherwise/', 'https://science.feedback.org/review/the-sun-cannot-explain-recent-global-warming-contrary-to-what-heartland-institute-report-claims/', 'https://science.feedback.org/review/climate-sensitivity-estimate-given-in-heartland-institutes-report-is-misleading/', 'https://science.feedback.org/review/cnn-interview-william-happer-incorrectly-claims-temperatures-dont-match-climate-model-projections/', 'https://science.feedback.org/review/cnn-interview-william-happer-misleads-impact-rising-carbon-dioxide-plant-life/', 'https://science.feedback.org/review/bloomberg-accurately-describes-rapidly-changing-arctic-eric-roston-blacki-migliozzi/', 'https://science.feedback.org/review/climate-context-for-a-wet-winter-in-drought-stricken-california-in-the-la-times-rong-gong-lin-ii-paige-st-john-los-angeles-times/', 'https://science.feedback.org/review/sea-level-rise-already-practical-concern-miami-bbc-story-accurately-explores-amanda-ruggeri-bbc/', 'https://science.feedback.org/review/one-of-the-most-troubling-ideas-about-climate-change-just-found-new-evidence-in-its-favor-chris-mooney-washington-post/', 'https://science.feedback.org/review/analysis-record-breaking-climate-change-pushes-world-uncharted-territory-damian-carrington-guardian/', 'https://science.feedback.org/review/great-barrier-reef-coral-climate-change-dieoff-the-new-york-times-damien-cave-justin-gillis/', 'https://science.feedback.org/review/why-are-climate-change-models-so-flawed-because-climate-science-is-so-incomplete-jeff-jacoby-boston-globe/', 'https://science.feedback.org/review/claim-new-scientist-humans-responsible-100-warming-mostly-correct/', 'https://science.feedback.org/review/science-shows-humans-primarily-responsible-climate-change-counter-epa-administrator-scott-pruitts-claim/', 'https://science.feedback.org/review/scientists-we-know-what-really-causes-climate-james-barrett-the-daily-wire/', 'https://science.feedback.org/review/heres-why-its-so-frickin-hot-right-now-andrew-freedman-mashable/', 'https://science.feedback.org/review/worlds-coral-reefs-severely-threatened-climate-change-human-impacts-abc-story-notes/', 'https://science.feedback.org/review/scientists-just-detected-major-change-earths-oceans-linked-warming-climate-chris-mooney-the-washington-post/', 'https://science.feedback.org/review/california-dam-crisis-says-changing-climate-noah-diffenbaugh-new-york-times/', 'https://science.feedback.org/review/big-melt-global-sea-ice-record-low-usa-today-doyle-rice/', 'https://science.feedback.org/review/daily-mail-inflates-disagreement-scientists-data-handling-make-unsupported-accusation-data-manipulation/', 'https://science.feedback.org/review/global-surface-temperatures-increasing-according-climate-projections-contrary-wall-street-journal-claim/', 'https://science.feedback.org/review/scientists-officially-declare-2016-hottest-year-record-chris-mooney-the-washington-post/', 'https://science.feedback.org/review/ocean-acidification-yet-another-wobbly-pillar-climate-alarmism-james-delingpole-the-spectator/', 'https://science.feedback.org/review/oceans-currently-acidifying-claims-contrary-contradict-observations/', 'https://science.feedback.org/review/arctic-ice-melt-already-affecting-weather-patterns-live-right-now-damian-carrington-the-guardian/', 'https://science.feedback.org/review/stunning-new-data-indicates-el-nino-drove-record-highs-global-temperatures-david-rose-daily-mail/', 'https://science.feedback.org/review/daily-mail-claim-2016-global-temperature-record-misleading/', 'https://science.feedback.org/review/great-barrier-reef-australia-coral-die-off-doyle-rice-usa-today/', 'https://science.feedback.org/review/higher-global-temperatures-sea-ice-record-lows-poles-brandon-miller-cnn/', 'https://science.feedback.org/review/the-phony-war-against-co2-the-wall-street-journal-rodney-nichols-harrison-schmitt/', 'https://science.feedback.org/review/sea-level-rise-accelerating-risk-associated-coastal-storm-surge-contrary-claim-forbes/', 'https://science.feedback.org/review/hillary-clinton-boards-climate-crisis-train-nowhere-roy-spencer-forbes/', 'https://science.feedback.org/review/sea-level-rise-those-non-disappearing-pacific-islands-bjorn-lomborg-wall-street-journal/', 'https://science.feedback.org/review/james-lovelock-interview-by-end-of-century-robots-will-have-taken-over-decca-aitkenhead-the-guardian/', 'https://science.feedback.org/review/climate-exaggeration-backfiring-robert-bradley-jr-forbes/', 'https://science.feedback.org/review/greenlands-huge-annual-ice-loss-is-even-worse-than-thought-damian-carrington-the-guardian/', 'https://science.feedback.org/review/flooding-of-coast-caused-by-global-warming-has-already-begun-justin-gillis-the-new-york-times/', 'https://science.feedback.org/review/sea-level-rise-much-1-2-meters-3-3-6-6-feet-year-2100/', 'https://science.feedback.org/review/arctic-will-be-ice-free-in-summer-next-year-robin-mckie-peter-wadhams-the-guardian/', 'https://science.feedback.org/review/louisiana-flooding-natural-disaster-weather-climate-change-oliver-milman-the-guardian/', 'https://science.feedback.org/review/environment-climate-change-records-broken-international-report-oliver-milman-the-guardian/', 'https://science.feedback.org/review/science-about-links-between-global-warming-and-massive-heat-waves-chris-mooney-washington-post/', 'https://science.feedback.org/review/heat-waves-u-s-already-become-frequent-intense-due-global-warming/', 'https://science.feedback.org/review/climate-change-emergency-jet-stream-shift-warning-global-warming-extreme-weather-gabriel-samuels-the-independent/', 'https://science.feedback.org/review/its-official-humans-are-making-the-earth-much-greener-washington-post-chris-mooney/', 'https://science.feedback.org/review/great-barrier-reef-perish-2030s-andrew-freedman-mashable/', 'https://science.feedback.org/review/bjorn-lomborg-overheated-climate-alarm-wall-street-journal/', 'https://science.feedback.org/review/washington-post-chris-mooneys-earth-no-parallel-66-million-years/', 'https://science.feedback.org/review/scientists-are-exaggerating-carbon-threat-to-reefs-and-marine-life/', 'https://science.feedback.org/review/sea-level-rise-global-warming-climate-change/', 'https://science.feedback.org/review/patrick-michaels-the-climate-snow-job-the-wall-street-journal/', 'https://science.feedback.org/review/james-taylor-2015-was-not-even-close-to-hottest-year-on-record/', 'https://science.feedback.org/review/analysis-justin-gillis-2015-hottest-year-global-warming/', 'https://science.feedback.org/review/analysis-of-chris-mooney-scientists-say-human-greenhouse-gas-emissions-have-canceled-the-next-ice-age/', 'https://science.feedback.org/review/analysis-of-matt-ridley-benny-peiser-your-complete-guide-to-the-climate-debate/', 'https://science.feedback.org/review/forbes-james-taylor-top-10-global-warming-lies-that-may-shock-you/', 'https://science.feedback.org/review/forbes-james-taylor-updated-nasa-data-polar-ice-not-receding-after-all/', 'https://science.feedback.org/review/andrew-freedman-mashable-hurricane-patricia-global-warming/', 'https://science.feedback.org/review/matthew-francis-forbes-even-without-drastic-predictions-coasts-are-in-danger/', 'https://science.feedback.org/review/andrew-freedman-south-carolina-flooding-climate-change/', 'https://science.feedback.org/review/betsy-mccaughey-wake-up-obama-climate-change-has-been-happening-forever/', 'https://science.feedback.org/review/tom-harris-deceptive-temperature-record-claims/', 'https://science.feedback.org/review/brad-plumer-el-nino-explained-why-this-years-could-be-one-of-the-strongest-on-record/', 'https://science.feedback.org/review/the-telegraph-christopher-booker-arctic-ice-has-made-fools-of-warmists/', 'https://science.feedback.org/review/the-telegraph-dan-hyde-earth-heading-for-mini-ice-age-within-15-years/', 'https://science.feedback.org/review/pope-francis-encyclical-laudato-si/', 'https://science.feedback.org/review/mashable-andrew-freedman-vanuatu-cyclone-pam-climate-change/', 'https://science.feedback.org/review/wall-street-journal-bjorn-lomborg-alarming-thing-climate-alarmism/']
verdicts = ['Inaccurate', 'Inaccurate', 'Incorrect', 'Misleading', 'Incorrect', 'Inaccurate', 'Incorrect', 'Misleading', 'Misleading', 'Inaccurate', 'Incorrect', 'Inaccurate', 'Misleading', 'Misleading', 'Inaccurate', 'Incorrect', 'Incorrect', 'Inaccurate', 'Inaccurate', 'Incorrect', 'Incorrect', 'Incorrect', 'Incorrect', 'Inaccurate', 'Inaccurate', 'Inaccurate', 'Flawed reasoning', 'Inaccurate', 'Flawed reasoning', 'Incorrect', 'Incorrect', 'Incorrect', 'Incorrect', 'Incorrect', 'Incorrect', 'Incorrect', 'Incorrect', 'Mostly accurate', '-1.75', '-1.4', 'Misleading', '1.3', 'Inaccurate', 'Inaccurate', 'Incorrect', 'Inaccurate', 'Inaccurate', '-1.8', '-1.7', 'Inaccurate', '-1.8', 'Accurate', 'Inaccurate', 'Inaccurate', 'Incorrect', '1', '1.5', 'Inaccurate', 'Incorrect', '-2', 'Inaccurate', 'Incorrect', '-2', 'Inaccurate', 'Incorrect', 'Accurate', 'Misleading', '-1', 'Misleading', 'Imprecise', '-1', 'Misleading', 'Inaccurate', 'Lacks context', '-1', 'Correct', 'Incorrect', '-1', 'Accurate', 'Incorrect', '-1', '-2', 'Incorrect', 'Incorrect', '0.3', 'Flawed reasoning', 'Imprecise', 'Incorrect', 'Incorrect', 'Accurate', 'Mostly accurate', 'Mostly accurate', 'Inaccurate', 'Flawed reasoning', 'Flawed reasoning', 'Flawed reasoning', 'Incorrect', 'Inaccurate', 'Inaccurate', 'Misleading', 'Inaccurate', 'Inaccurate', 'Misleading', 'Incorrect', 'Inaccurate', 'Incorrect', 'Misleading', 'Inaccurate', 'Inaccurate', 'Incorrect', 'Inaccurate', '1.7', '-2', 'Inaccurate', '-0.8', 'Incorrect', 'Inaccurate', 'Misleading', 'Inaccurate', '-1.7', '-1.8', 'Incorrect', 'Incorrect', 'Inaccurate', 'Inaccurate', 'Incorrect', '-1.6', 'Incorrect', 'Misleading', 'Flawed reasoning', 'Incorrect', 'Incorrect', 'Inaccurate', 'Inaccurate', '1.3', 'Inaccurate', 'Incorrect', 'Incorrect', 'Incorrect', '1.7', 'Inaccurate', 'Inaccurate', '-1', '-1.4', '-2', 'Accurate', 'Flawed reasoning', 'Incorrect', 'Inaccurate', '-1.9', 'Flawed reasoning', 'Inaccurate', 'Misleading', '-2', 'Lacks context', '1.5', '0.2', '-0.3', '1.0', '1.5', '2', '2', '-2', '1.3', 'Incorrect', '1', '1.5', 'Inaccurate', '0.7', '-1.5', '1.3', 'Incorrect', 'Misleading', '-2', '1', 'Inaccurate', '1.7', '1.5', 'Incorrect', '0.8', '-1', '1.3', '1.8', '1', '-1.8', '-0.2', '-1.3', '-2', 'Inaccurate', '2', '-1.7', '1.5', 'Inaccurate', '0.3', '0.3', '1.7', '1.3', '-1.5', '0.7', '2', '-0.5', '-1.4', '-0.6', 'Incorrect', '0.8', '0.5', '-0.7', '-2', '0.7', 'Inaccurate', '1.4', 'Incorrect', '-1.7', '2', 'Incorrect', 'Mostly correct', '1.5', '1.6', 'Inaccurate', 'Inaccurate', '1', '-1.5', '1', '-0.7', '-1.8', 'Incorrect', '1.3', '1', 'Incorrect', '-2', 'Inaccurate', '-2', '-2', 'Misleading', 'Incorrect', 'Inaccurate', 'Flawed reasoning', '1.2', 'Incorrect', '0.3', 'Incorrect', '-2', '-2', 'Inaccurate', 'Incorrect', 'Misleading', 'Incorrect', 'Misleading', '2', '0.8', '1.7', '1.25', '1.5', '1.8', '-2', 'Mostly correct', 'Incorrect', '-1.5', '1.3', 'Mostly correct', '2', '1', '1.4', 'Unsupported', 'Misleading', '1.9', '-2', 'Inaccurate', '0.6', '-1.9', 'Misleading', '1.6', '2', '-2', 'Inaccurate', '-1.8', '-1.2', '-1.4', '-1.7', '1.6', '1.7', 'Correct', '-0.6', '1.3', '1.3', '1.7', 'Accurate', '-1.4', '0.6', '1.0', '-1.3', '1.5', '-2', '1.7', '-1.8', '-1.8', '1.9', '1.7', '-1.5', '-2', '-2', '1.4', '1', '1.3', '-2', '-1.9', '1.7', '-2', '-1.5', '0.9', '1.5', '-1.6']
sources = ['Facebook, YouTube, TikTok, X/Twitter, 2024-03-20',
 'Facebook, 2024-06-06',
 'Facebook, YouTube, TikTok, X/Twitter, 2024-05-11',
 'PragerU, Judith Curry, 2024-04-15',
 'Daily Sceptic, Chris Morrison, 2024-04-08',
 'Daily Sceptic, Stephen Andrews, 2024-03-08',
 'The Epoch Times, Allan Astrup Jensen, 2024-02-06',
 'Bright Insight, Jimmy Corsetti, 2024-02-04',
 'Facebook, The Atlas Society, Social media users, 2024-01-26',
 'Facebook, YouTube, Social media users, 2024-01-12',
 'Tucker Carlson Network, Willie Soon, 2024-01-09',
 'Twitter/X, Peter Imanuelsen (aka Peter Sweden), 2023-11-27',
 'Daily Mail, Jonathan Chadwick, 2023-12-13',
 'Instagram, Social media users, 2023-12-19',
 'Facebook, Facebook users, 2023-12-09',
 'Climate Change is Crap, Facebook users, 2023-11-18',
 'Conservative Political Action Conference, Ian Plimer, 2023-10-03',
 'Twitter/X, Steve Milloy, 2023-11-18',
 'The Wall Street Journal, Holman W. Jenkins, 2023-11-03',
 'Twitter/X, Social media users, 2023-09-02',
 'Clintel, The Epoch Times, Benoit Rittaud, Naveen Athrappully, Viv Forbes, Richard Lindzen, Patrick Moore, Ian Plimer, Guus Berkhout, Christopher Monckton, 2023-08-19',
 'Facebook, Facebook users, 2023-08-07',
 'The Joe Rogan Experience, TikTok, Spotify, Joe Rogan, Jimmy Corsetti, 2023-01-18',
 'Media Research Center, Ian Plimer, 2023-05-02',
 'CFACT, Instagram, Don Easterbrook, Judith Curry, 2023-04-16',
 'Facebook, William Deagle, 2023-03-18',
 'Instagram, Social media users, 2023-01-17',
 'Fox News, Tom Harris, 2022-10-12',
 'The Joe Rogan Experience, Joe Rogan, 2022-10-02',
 'The Epoch Times, Patrick Moore, 2022-09-06',
 'Climate Change is Crap, Richard Lindzen, 2022-08-30',
 'Daily Sceptic, Toby Young, 2022-08-18',
 'CBN News, Jerome Corsi, 2022-07-29',
 'Suspicious0bservers, Ben Davidson, 2022-07-14',
 'Daily Sceptic, Chris Morrison, 2022-05-19',
 'Watts Up With That?, Christopher Monckton, 2022-04-04',
 'CO2 Coalition, Andy May, 2022-03-03',
 'The Washington Post, Sarah Kaplan, 2022-03-07',
 'The Wall Street Journal, by Steve Koonin, on 2022-02-17.',
 'PragerU, by Steve Koonin, on 2024-10-25.',
 'The Daily Wire, Ben Zeisloft, 2021-10-24',
 'CNN, by Rachel Ramirez, on 2021-08-19.',
 'Facebook, Facebook users, 2021-09-05',
 'Climate Change Dispatch, Climate Realism, Watts Up With That?, David Mason-Jones, 2021-08-28',
 'Newsmax, The Epoch Times, The Western Journal, Alex Newman, 2021-08-16',
 'Climate Change Dispatch, The Australian, Watts Up With That?, Peter Ridd, 2021-07-21',
 'Suspicious0bservers, Ben Davidson, 2021-05-05',
 'The Washington Times, CO2 Coalition, by Gregory Wrightstone, on 2021-04-21.',
 'The Wall Street Journal, by Mark P. Mills, on 2021-04-25.',
 'Daily Mail, Emily Crane, 2021-04-22',
 'Stossel TV, by David Legates, Patrick Michaels, John Stossel, on 2021-04-17.',
 'CNN, Amy Woodyatt, 2021-04-09',
 'Turning Point USA, Candace Owens, Charlie Kirk, 2021-04-11',
 'The Guardian, Steve Rose, Ties Van der Hoeven, 2021-03-20',
 'Geoengineering Watch, Dane Wigington, 2021-03-10',
 'National Geographic, by Craig Welch, on 2021-03-11.',
 'The New York Times, by Jeremy White, Moises Velasquez-Manoff, on 2021-03-02.',
 'Facebook, Facebook users, 2021-02-20',
 'Gaia, Anonymous, 2020-12-11',
 'Breitbart, by James Delingpole, Indur Goklany, on 2021-02-07.',
 'Facebook, Facebook users, 2021-01-29',
 'PragerU, William Happer, 2021-01-24',
 'Sky News Australia, Daily Mail, by Ian Plimer, on 2021-01-13.',
 'NoTricksZone, Watts Up With That?, Kenneth Richard, Pierre L. Gosselin, 2021-01-11',
 'Instagram, Patrick Moore, 2020-12-16',
 'Instagram, Patagonia, 2020-11-30',
 'Facebook, Facebook users, 2018-11-21',
 'The Guardian, by Jonathan Watts, on 2020-10-27.',
 'Climate Realism, Heartland Institute, Andy Singer, 2020-10-23',
 'The Guardian, Fiona Harvey, 2020-10-05',
 'The Daily Caller, by Chris White, on 2020-09-13.',
 'Facebook, Facebook users, 2020-09-09',
 'We The Governed, Glen Morgan, 2020-09-10',
 '8 Billion Trees, Pamela Anderson, 2020-08-23',
 'CNN, by Brandon Miller, Max Claypool, on 2020-08-14.',
 'The New York Times, Henry Fountain, 2020-07-20',
 'Instagram, Otis Holland, 2020-07-11',
 'Forbes, by Michael Shellenberger, on 2020-06-28.',
 'The New York Times, Henry Fountain, 2020-06-29',
 'Clear Energy Alliance, Mark Mathis, 2020-06-10',
 'The Guardian, by Jonathan Watts, on 2020-06-13.',
 'PragerU, by Richard Lindzen, on 2020-05-08.',
 'The Sun, Chris Pollard, 2020-05-13',
 'PragerU, Anonymous, 2020-05-05',
 'Business Insider, by Sarah Al-Arshani, on 2020-05-05.',
 'Tichys Einblick, Holger Douglas, 2020-04-12',
 'Express, Sebastian Kettley, 2020-02-18',
 'I Love Carbon Dioxide, Cole Ryan, 2020-04-12',
 'CFACT, Jay Lehr, 2020-04-08',
 'ABC News, Michael Slezak, Penny Timms, 2020-04-06',
 'The Independent, Louise Boyle, 2020-03-26',
 'Now This, Lani Chan, Tom McKenna, 2020-03-20',
 'Watts Up With That?, Willis Eschenbach, 2020-03-08',
 'LifeSite News, Andrea Widberg, 2020-03-17',
 'Electroverse, Cap Allon, 2020-01-22',
 'Natural News, Mike Adams, 2013-06-22',
 'Electroverse, Roger Higgs, 2020-03-11',
 'American Thinker, Thomas Lifson, 2020-03-07',
 'Principia Scientific, IceAgeNow, 2016-11-26',
 'PragerU, Patrick Moore, 2020-01-21',
 'Breitbart, Signs of the Times, James Delingpole, Vicki Batts, 2019-05-28',
 "Investors' Business Daily, Anonymous, 2018-03-29",
 'Issues & Insights, I&I Editorial Board, 2019-08-26',
 'Express, The Sun, Harry Pettit, Sean Martin, 2020-02-02',
 'Technocracy.news, Tim Ball, 2018-09-13',
 'Facebook, Anonymous, 2020-01-02',
 'Breitbart, NewsPunch, Paul Joseph Watson, Sean Adl-Tabatabai, Thomas D. Williams, 2020-01-05',
 'Principia Scientific, James Edward Kamis, 2018-11-07',
 'NewsPunch, Principia Scientific, Your News Wire, Baxter Dmitry, 2018-07-08',
 'Facebook, Anonymous, 2019-12-11',
 'Facebook, Anonymous, 2019-12-11',
 'Australian Broadcasting Corporation, by Joshua Byrd, Nathan Hoad, Tim Leslie, on 2019-12-05.',
 'The Australian, by Ian Plimer, on 2019-11-22.',
 'Climate Change Dispatch, James Edward Kamis, 2018-08-07',
 'The New York Times, by Eugene Linden, on 2019-11-08.',
 'Daily Mail, Fox News, Chris Ciaccia, Stacy Liberatore, 2019-10-31',
 'Facebook, Anonymous, 2019-08-19',
 'Breitbart, Thomas D. Williams, 2019-09-24',
 'Facebook, Anonymous, 2018-10-12',
 'The Telegraph, by Sarah Knapton, on 2019-10-15.',
 "Clintel, by Ingemar Nordin, Viv Forbes, Terry Dunleavy, Rob Lemeire, Richard Lindzen, Reynald du Berger, Morten Jodal, Jim O'Brien, Jeffrey Fos, Guus Berkhout, Fritz Vahrenholt, Christopher Monckton, Benoit Rittaud, Alberto Prestininzi, on 2019-09-23.",
 'The New Yorker, Jonathan Franzen, 2019-09-08',
 'Townhall, Kurt Schlichter, 2019-09-09',
 'TFP Student Action, Marc Morano, 2019-08-13',
 'Powerlineblog, Real Clear Energy, WND, James Taylor, John Hinderaker, 2019-08-29',
 'Natural News, Ethan Huff, 2019-08-30',
 'The Washington Examiner, Zero Hedge, by Caleb Stewart Rossiter, Patrick Michaels, on 2019-08-25.',
 'BBC News, Roger Hallam, 2019-08-17',
 'Fox News, Rick Perry, 2019-08-13',
 'Ice Age Now, Robert W. Felix, William Happer, 2017-02-19',
 'Electroverse, Cap Allon, Nir Shaviv, 2019-08-11',
 'Express, Sean Martin, 2019-07-29',
 'joannenova.com.au, Joanne Nova, 2019-07-26',
 'EXOMATRIXTV, Tony Heller, 2019-07-20',
 'NBC News, by Jaclyn Jeffrey-Wilensky, on 2019-07-24.',
 'Townhall, Gregory Rummo, 2019-07-23',
 'Sky News Australia, Alan Jones, Nils Axel-Mörner, 2019-06-17',
 'Zero Hedge, Summit.news, Sky News Australia, Fox News, Paul Joseph Watson, Pekka Malmi, Jyrki Kauppinen, 2019-07-11',
 'Fox News, Patrick Michaels, 2018-10-21',
 'The Independent, by Conrad Duncan, on 2019-07-02.',
 'The New American, James Murphy, Tony Heller, 2019-06-26',
 'Infowars, Zero Hedge, Anonymous, 2019-04-22',
 'Financial Post, by Ross McKitrick, on 2019-06-07.',
 'IFLScience, by James Felton, on 2019-06-04.',
 'The Australian, by Ian Plimer, on 2019-05-16.',
 'NBC News, Denise Chow, 2019-05-14',
 '"Ivanka_Trump" Facebook account, Dan Peña, 2019-05-04',
 'Derek Utley, John Coleman, 2019-02-22',
 'Express, Sean Martin, 2019-04-15',
 'Breitbart, by John Nolte, on 2019-04-09.',
 'The Blaze, Chris Enloe, 2019-03-26',
 'The Hill, Caleb Rossiter, 2019-03-06',
 'Facebook, Bernie Sanders, 2019-03-04',
 'The Western Journal, by Jay Lehr, Tom Harris, on 2019-02-20.',
 "Financial Times, Barclay's, 2019-02-19",
 'The New York Times, by Coral Davenport, on 2018-10-07.',
 'The Guardian, by Damian Carrington, on 2018-05-31.',
 'E&E News, Science Magazine, by Scott Waldman, on 2018-05-17.',
 'BBC, by Matt McGrath, on 2018-10-08.',
 'CNN, by Brandon Miller, on 2018-10-08.',
 'USA Today, by Doyle Rice, on 2018-05-17.',
 'The Washington Post, by Jason Samenow, on 2018-02-26.',
 'YouTube, Friends of Science, by Steve Goreham, on 2017-07-01.',
 'CNN, Fox News, by Brandon Miller, on 2018-02-26.',
 'Metro, Jasper Hamill, 2018-11-16',
 'The Washington Post, by Brady Dennis, Chris Mooney, on 2018-10-31.',
 'The Guardian, by Damian Carrington, on 2018-10-10.',
 'BBC, Myron Ebell, 2018-10-08',
 'The Guardian, by Oliver Milman, on 2018-10-04.',
 'USA Today, by Roy Spencer, on 2018-09-14.',
 'The Guardian, by Jonathan Watts, on 2018-08-21.',
 'The Australian, Ian Plimer, 2018-08-07',
 'The Australian, Ian Plimer, 2018-08-07',
 'The Australian, by Ian Plimer, on 2018-08-07.',
 'The Guardian, by Jonathan Watts, on 2018-08-07.',
 'Twitter/X, Donald Trump, 2018-08-05',
 'The Washington Post, by Angela Fritz, Joel Achenbach, on 2018-07-26.',
 'NPR, by Nathan Rott, on 2018-07-23.',
 'The Wall Street Journal, Fred Singer, 2018-05-15',
 'USA Today, by Doyle Rice, on 2018-07-06.',
 'Financial Post, by Ross McKitrick, on 2018-06-20.',
 'The Washington Post, by Chris Mooney, on 2018-06-13.',
 'National Geographic, by Craig Welch, on 2018-06-06.',
 'The New York Times, by Henry Fountain, on 2018-05-24.',
 'The Wall Street Journal, by Fred Singer, on 2018-05-15.',
 'Business Insider, by Kevin Loria, on 2018-05-08.',
 'The Australian, by Graham Lloyd, on 2018-04-19.',
 "Investors' Business Daily, by Anonymous, on 2018-04-09.",
 'The Rush Limbaugh Show, Rush Limbaugh, 2018-04-02',
 'The Washington Post, by Chris Mooney, on 2018-03-19.',
 'Financial Post, by Susan Crockford, on 2018-02-27.',
 'CNN, by Brandon Miller, on 2018-02-13.',
 'ITV, Donald Trump, 2018-01-28',
 'National Geographic, by Sarah Gibbens, on 2017-12-07.',
 'The Atlantic, by James Hamblin, on 2017-08-02.',
 'The New York Times, by Jugal K. Patel, Justin Gillis, on 2017-07-12.',
 'The New York Times, by Henry Fountain, on 2017-03-22.',
 'Conservative Tribune, by Benjamin Arie, on 2017-06-22.',
 'IFLScience, by Robin Andrews, on 2017-08-11.',
 'The New York Times, by Justin Gillis, on 2017-01-18.',
 'Futurism, by Jolene Creighton, June Javelosa, on 2017-04-12.',
 'Breitbart, by Joel Pollak, on 2017-12-09.',
 'The Daily Caller, by Michael Bastasch, on 2017-11-29.',
 'The Daily Caller, John Christy, 2017-11-29',
 'Grist, by Eric Holthaus, on 2017-11-21.',
 'The Guardian, by Ruth Michaelson, Richard Luscombe, Niko Kommenda, Justin McCurry, Josh Holder, Jonathan Watts, Helen Roxburgh, Dom Phillips, on 2017-11-03.',
 'The Independent, by Andrew Griffin, on 2017-10-26.',
 'The Australian, by Ian Plimer, on 2017-10-23.',
 'BBC, by Roger Harrabin, on 2017-10-23.',
 'Global Warming Policy Foundation, Tony Abbott, 2017-10-09',
 'The New York Times, by Justin Gillis, on 2017-09-19.',
 'The Daily Wire, James Barrett, 2017-09-19',
 'The Daily Wire, by James Barrett, on 2017-09-19.',
 'The Atlantic, by Robinson Meyer, on 2017-09-07.',
 'The Spectator, Jennifer Marohasy, 2017-08-21',
 'The Atlantic, Robinson Meyer, 2017-08-27',
 'The Atlantic, by Robinson Meyer, on 2017-08-27.',
 'The New York Times, by Henry Fountain, on 2017-08-23.',
 'BBC, Nigel Lawson, 2017-08-10',
 'BBC, Nigel Lawson, 2017-08-10',
 'Forbes, by Trevor Nace, on 2017-08-03.',
 'The Wall Street Journal, by David Henderson, John Cochrane, on 2017-07-30.',
 'Think Progress, by Joe Romm, on 2017-07-19.',
 'New York Magazine, by David Wallace-Wells, on 2017-07-09.',
 'Breitbart, by James Delingpole, on 2017-07-09.',
 'BBC, Stephen Hawking, 2017-07-02',
 'The Atlantic, by Robinson Meyer, on 2017-06-29.',
 'The New York Times, by Brad Plumer, Nadja Popovich, on 2017-06-22.',
 'CNBC, Rick Perry, 2017-06-19',
 'Breitbart, by James Delingpole, on 2017-06-06.',
 'Breitbart, James Delingpole, 2017-06-06',
 'Breitbart, by Thomas D. Williams, on 2017-06-05.',
 'CNN, by Senator Ted Cruz, on 2017-05-29.',
 'Heartland Institute, Craig Idso, Fred Singer, Robert Carter, 2024-08-14',
 'Heartland Institute, Craig Idso, Fred Singer, Robert Carter, 2024-08-14',
 'Heartland Institute, Craig Idso, Fred Singer, Robert Carter, 2024-08-14',
 'Heartland Institute, Craig Idso, Fred Singer, Robert Carter, 2024-08-14',
 'The New York Times, by Justin Gillis, on 2017-05-18.',
 'Heartland Institute, Craig Idso, Fred Singer, Robert Carter, 2024-08-14',
 'Think Progress, by Joe Romm, on 2017-05-16.',
 'Heartland Institute, Craig Idso, Fred Singer, Robert Carter, 2024-08-14',
 'The Daily Wire, by Joseph Curl, on 2017-05-08.',
 'The Telegraph, by Christopher Booker, on 2017-05-06.',
 'Heartland Institute, Craig Idso, Fred Singer, Robert Carter, 2024-08-14',
 'Heartland Institute, Craig Idso, Fred Singer, Robert Carter, 2024-08-14',
 'Heartland Institute, Craig Idso, Fred Singer, Robert Carter, 2024-08-14',
 'CNN, William Happer, 2017-04-22',
 'CNN, William Happer, 2017-04-22',
 'Bloomberg, by Blacki Migliozzi, Eric Roston, on 2017-04-19.',
 'Los Angeles Times, by Paige St. John, Rong-Gong Lin II, on 2017-04-12.',
 'BBC, by Amanda Ruggeri, on 2017-04-04.',
 'The Washington Post, by Chris Mooney, on 2017-03-27.',
 'The Guardian, by Damian Carrington, on 2017-03-20.',
 'The New York Times, by Damien Cave, Justin Gillis, on 2017-03-15.',
 'The Boston Globe, by Jeff Jacoby, on 2017-03-14.',
 'New Scientist, Michael Le Page, 2017-03-08',
 'CNBC, Scott Pruitt, 2017-03-09',
 'The Daily Wire, by James Barrett, on 2017-02-24.',
 'Mashable, by Andrew Freedman, on 2017-02-24.',
 'ABC News, Greg Hassall, Rebecca Latham, 2017-02-23',
 'The Washington Post, by Chris Mooney, on 2017-02-15.',
 'The New York Times, by Noah Diffenbaugh, on 2017-02-14.',
 'USA Today, by Doyle Rice, on 2017-02-08.',
 'Daily Mail, David Rose, 2017-02-04',
 'The Wall Street Journal, Wall Street Journal editorial, 2017-01-19',
 'The Washington Post, by Chris Mooney, on 2017-01-18.',
 'The Spectator, by James Delingpole, on 2016-04-30.',
 'The Spectator, James Delingpole, Mike Wallace, 2016-04-30',
 'The Guardian, by Damian Carrington, on 2016-12-19.',
 'Daily Mail, by David Rose, Judith Curry, on 2016-11-26.',
 'Daily Mail, David Rose, 2016-11-26',
 'USA Today, by Doyle Rice, on 2016-11-29.',
 'CNN, by Brandon Miller, on 2016-11-19.',
 'The Wall Street Journal, by Harrison Schmitt, Rodney Nichols, on 2016-10-31.',
 'Forbes, Roy Spencer, 2016-10-25',
 'Forbes, by Roy Spencer, on 2016-10-25.',
 'The Wall Street Journal, by Bjorn Lomborg, on 2016-10-13.',
 'The Guardian, by Decca Aitkenhead, James Lovelock, on 2016-09-30.',
 'Forbes, by Robert Bradley Jr., on 2016-09-23.',
 'The Guardian, by Damian Carrington, on 2016-09-23.',
 'The New York Times, by Justin Gillis, on 2016-09-03.',
 'The New York Times, Justin Gillis, 2016-09-03',
 'The Observer, The Guardian, by Robin McKie, Peter Wadhams, on 2016-08-21.',
 'The Guardian, by Oliver Milman, on 2016-08-16.',
 'The Guardian, by Oliver Milman, on 2016-08-02.',
 'The Washington Post, by Chris Mooney, on 2016-07-21.',
 'The Washington Post, Chris Mooney, 2016-07-21',
 'The Independent, by Gabriel Samuels, on 2016-06-30.',
 'The Washington Post, by Chris Mooney, on 2016-06-27.',
 'Mashable, by Andrew Freedman, on 2016-04-29.',
 'The Wall Street Journal, by Bjorn Lomborg, on 2016-04-06.',
 'The Washington Post, by Chris Mooney, on 2016-03-21.',
 'The Australian, The Times, by Ben Webster, on 2016-03-01.',
 'The New York Times, by Justin Gillis, on 2016-02-22.',
 'The Wall Street Journal, by Patrick Michaels, on 2016-01-24.',
 'Forbes, by James Taylor, on 2016-01-14.',
 'The New York Times, by Justin Gillis, on 2016-01-20.',
 'The Washington Post, by Chris Mooney, on 2016-01-13.',
 'The Wall Street Journal, by Matt Ridley, Benny Peiser, on 2015-11-27.',
 'Forbes, by James Taylor, on 2015-02-09.',
 'Forbes, by James Taylor, on 2015-05-19.',
 'Mashable, by Andrew Freedman, on 2015-10-23.',
 'Forbes, by Matthew Francis, on 2015-07-28.',
 'Mashable, by Andrew Freedman, on 2015-10-06.',
 'New York Post, by Betsy McCaughey, on 2015-09-07.',
 'The Washington Times, by Tom Harris, on 2015-08-23.',
 'Vox, by Brad Plumer, on 2015-08-17.',
 'The Telegraph, by Christopher Booker, on 2015-07-25.',
 'The Telegraph, by Dan Hyde, on 2015-07-11.',
 'Vatican.va, by Pope Francis, on 2015-06-18.',
 'Mashable, by Andrew Freedman, on 2015-03-16.',
 'The Wall Street Journal, by Bjorn Lomborg, on 2015-02-01.']

In [46]:
# @title Consolidating the heading variations to use as the csv's column labels **(RUN)**
h_dict = {
    'Key takeaway': 'key_takeaway',
    'Summary': 'summary',
    'Reviewed content': 'reviewed_content',
    'Verdict detail': 'verdict_detail',
    'Full Claim': 'full_claim',
    'Scientists’ Feedback': 'review',
    'Review': 'review',
    'Reviewers’ Feedback': 'review',
    'Annotations': 'review',
    'Reviewers’ Overall Feedback': 'review',
    'Scientist’s Feedback': 'review',
    'Featured Annotations': 'review',
    'Reviewers’ Comments': 'review',
    'GUEST COMMENTS': 'review',
    'REVIEWERS’ OVERALL FEEDBACK': 'review',
    'Featured Annotations\xa0': 'review',
    'Key Take-aways': 'key_takeaway',
    'Key Takeaways': 'key_takeaway',
    'SUMMARY': 'summary',
    'Key Take-away': 'key_takeaway',
    'CLAIM 1 (Inaccurate):': 'unique',
    'CLAIM 2 (Incorrect):': 'unique',
    'UPDATES': 'unique',
    'Conclusion': 'unique',
    'References': 'unique',
    'REFERENCES': 'references'
}
keys = list(h_dict.keys())
keys = sorted(keys, key=len)[::-1]
print("keys:", keys)
vals = list(set(h_dict.values()) - set(["unique", "reviewed_content", "summary"]))
vals.insert(0, "claim")
vals.insert(1, "headline")
print("output file headings:", vals)
df = pd.DataFrame({"url": urls, "verdict": verdicts, "source": sources})
df

keys: ['REVIEWERS’ OVERALL FEEDBACK', 'Reviewers’ Overall Feedback', 'CLAIM 1 (Inaccurate):', 'Featured Annotations\xa0', 'CLAIM 2 (Incorrect):', 'Featured Annotations', 'Scientist’s Feedback', 'Scientists’ Feedback', 'Reviewers’ Comments', 'Reviewers’ Feedback', 'Reviewed content', 'Key Take-aways', 'GUEST COMMENTS', 'Verdict detail', 'Key Take-away', 'Key Takeaways', 'Key takeaway', 'Annotations', 'REFERENCES', 'References', 'Conclusion', 'Full Claim', 'UPDATES', 'SUMMARY', 'Summary', 'Review']
output file headings: ['claim', 'headline', 'verdict_detail', 'key_takeaway', 'full_claim', 'references', 'review']


,url,verdict,source
0,https://science.feedback.org/review/pyramid-sh...,Inaccurate,"Facebook, YouTube, TikTok, X/Twitter, 2024-03-20"
1,https://science.feedback.org/review/sea-levels...,Inaccurate,"Facebook, 2024-06-06"
2,https://science.feedback.org/review/colorful-a...,Incorrect,"Facebook, YouTube, TikTok, X/Twitter, 2024-05-11"
3,https://science.feedback.org/review/no-evidenc...,Misleading,"PragerU, Judith Curry, 2024-04-15"
4,https://science.feedback.org/review/carbon-iso...,Incorrect,"Daily Sceptic, Chris Morrison, 2024-04-08"
...,...,...,...
307,https://science.feedback.org/review/the-telegr...,-2,"The Telegraph, by Christopher Booker, on 2015-..."
308,https://science.feedback.org/review/the-telegr...,-1.5,"The Telegraph, by Dan Hyde, on 2015-07-11."
309,https://science.feedback.org/review/pope-franc...,0.9,"Vatican.va, by Pope Francis, on 2015-06-18."
310,https://science.feedback.org/review/mashable-a...,1.5,"Mashable, by Andrew Freedman, on 2015-03-16."


In [90]:
# @title Extracting scientist reviews (in the format of a DICTIONARY where the KEY is the heading and the VALUE is the paragraph text under that heading)

import re
import pandas as pd
import csv

labels = ["Accurate", "Correct", "Mostly accurate", "Mostly correct", "Lacks context", "Imprecise", "Partially correct", "Unsupported", "Misleading", "Inaccurate", "Incorrect", "Flawed reasoning", "Factually inaccurate", "Cherry-picking", ]

def replaceList(s, l):
  for i in l:
    s = s.replace(i, "")
  return s

def getReview(soup, is_cred_score):
  # Gets all text from "Review section" - ResultSet object with 1 element (all text) is returned (indexing works because it's similar to a list)
  info = soup.find_all("div", class_="wp-block-group is-layout-constrained")

  #Finding all headings
  heads = info[0].find_all("h2")
  if heads == []:
    heads = [i for i in info[0].find_all("h2")]

  headings = []
  h = ""
  h2 = ""
  h3 = ""
  for i in heads:
    h1 = i.text.strip("\n")
    if h1 not in keys:
      continue
    if h_dict[h1] == "key_takeaway":
      h = h1
    if h1.lower() == "full claim":
      h2 = h1
    if h1.lower() == "verdict detail":
      h3 = h1
    #Only keep known headings (not "unique" ones)
    if h1.lower() != "reviewed content" and h_dict[h1] != "unique":
      headings.append(h1)

  paragraphs = []
  text = soup.get_text()
  #Can't split based on "\n" so splitting after substituting different character for newline
  paragraphs = [i for i in re.sub("\n", "~*~", text).split("~*~") if i != "" and i != " "]
  review_content = {}
  c_ind = -2
  if not is_cred_score:
    #Getting the key takeaway and verdict detail
    if h:
      h_ind = paragraphs.index(h)
      review_content['key_takeaway'] = paragraphs[h_ind+1]
      headings.remove(h)
    if h3:
      verd_det = replaceList(info[0].find_all("div", class_="wp-block-group is-layout-flow wp-block-group-is-layout-flow")[1].text, ["\r", "\t", "Verdict detail", "See our method of evaluation"]).replace("\n\n", "").replace("\n", " ").replace("::", ":")
      for lab in labels:
        if lab in verd_det and "Correct but..." not in verd_det:
          ind = verd_det.index(lab) + len(lab)
          if verd_det[ind] != ":":
            space = " "
            if verd_det[ind] == space:
              space = ""
            verd_det = verd_det[:ind] + ":" + space + verd_det[ind:]
      re.sub(":", lambda ele: ele[0] + " ", verd_det)
      if verd_det[0] == " ":
        verd_det = verd_det[1:]
      review_content['verdict_detail'] = verd_det
      headings.remove(h3)
    #Getting the claim and full claim
    c_ind = paragraphs.index("Claim:")
    review_content['claim'] = paragraphs[c_ind+1]
    if h2:
      c_ind = paragraphs.index(h2)
      review_content['full_claim'] = paragraphs[c_ind + 1]
      headings.remove(h2)
  else:
    #Getting the article headline from the post title (posts with credibility score have the "headline" instead of the "claim")
    h_ind = 0
    for ind, p in enumerate(paragraphs):
      if "Headline: " in p:
        h_ind = ind
        break
    if h_ind:
      review_content['headline'] = paragraphs[h_ind][10:]
    else:
      review_content['headline'] = ' '.join(paragraphs[0].split(" of ")[1:]).replace(" - Science Feedback", "")

  #We now want to skip content until the second date appears (first one is when the review was published, second date is the date of claim publication)
  #"start" will be 0 for posts with credibility scores. If the post has verdicts, the starting paragraph will be 2 below the "Full Claim" heading
  start, end, heading_indices = c_ind+2, len(paragraphs)-1, []
  c = 0
  if is_cred_score:
    for h in headings:
      if h_dict[h] == "review":
        start = paragraphs.index(h)
        break
  for ind, p in enumerate(paragraphs):
    if "Science Feedback is a non-partisan, non-profit organization dedicated to science education." in p and end == len(paragraphs)-1:
      end = ind
  paragraphs = paragraphs[start:end]

  #Extracting all scientists' titles and adding a ": " after the title
  scientist_title = info[0].find_all("p", class_="wp-block-sciencefeedback-core-member__subtitle")
  scientist_title = [i.text for i in scientist_title]
  for ind, p in enumerate(paragraphs):
    for t in scientist_title:
      if t in p:
        i = p.index(t)
        paragraphs[ind] = p[:i] + " " + p[i:i+len(t)] + ":" + p[i+len(t):]

  rev = replaceList(' '.join(paragraphs), ["Scientists’ Feedback SUMMARY", "\xa0", "  "])
  if rev[0] == " ":
    rev = rev[1:]

  #putting a colon and space after every heading to distinguish it from the review (e.g: "Scientists' Feedback..." --> "Scientists' Feedback: ...")
  for k in keys:
    if k in rev:
      add_ind = rev.index(k) + len(k)
      #"Reviewers' Overall Feedback" would become "Review: ers' Overall Feedback: " --> don't want to add colon if the heading is "Review", and there are more characters after it (i.e. there is no space right after "Review")
      if k == "Review" and rev[add_ind] != " ":
        continue
      n = rev[add_ind:add_ind+2]
      if ":" in n and " " not in n:
        rev = rev[:add_ind+1] + " " + rev[add_ind+1:]
      elif " " in n and ":" not in n:
        rev = rev[:add_ind] + ":" + rev[add_ind:]
      elif ":" not in n and " " not in n:
        rev = rev[:add_ind] + ": " + rev[add_ind:]

  #Extracting References if present
  if " REFERENCES " in rev:
    rev = rev.split(" REFERENCES ")
    review_content['review'] = rev[0]
    review_content['references'] = rev[1]
  elif " REFERENCES: " in rev:
    rev = rev.split(" REFERENCES: ")
    review_content['review'] = rev[0]
    review_content['references'] = rev[1]
  else:
    review_content['review'] = rev

  return review_content

h_content = {i: [] for i in vals}
#Loop through each url
for ind, sub_url in enumerate(urls):
  html = requests.get(sub_url)
  soup = BeautifulSoup(html.text, 'html.parser')
  is_cred_score = False
  #all credibility scores are between length 1 and 5 (e.g: 1, -1.75)
  if len(verdicts[ind]) < 6:
    is_cred_score = True
  review_content = getReview(soup, is_cred_score)
  #If a key is unused, set its value as "None"
  used_keys = set()
  for key, value in review_content.items():
    h_content[key].append(value)
    used_keys.add(key)
  unused = list(set(vals) - used_keys)
  for i in unused:
    h_content[i].append(None)
#Append new columns to dataframe
for key, value in h_content.items():
  df[key] = value

In [91]:
df.to_csv("science_feedback_data.csv", index=False)
df

,url,verdict,source,claim,headline,verdict_detail,key_takeaway,full_claim,references,review
0,https://science.feedback.org/review/pyramid-sh...,Inaccurate,"Facebook, YouTube, TikTok, X/Twitter, 2024-03-20",Ancient civilizations once inhabited Antarctic...,None,Factually inaccurate: There is no evidence of ...,The rocky pyramid-shaped feature in Antarctica...,Thawing ice in Antarctica is revealing pyramid...,1- Hess (2016) McKnight’s Physical Geography: ...,"On 20 March 2024, a video on YouTube claimed t..."
1,https://science.feedback.org/review/sea-levels...,Inaccurate,"Facebook, 2024-06-06",Sea-level rise is not occurring or showing any...,None,Inaccurate: Scientific evidence unambiguously ...,Scientists have shown that global mean sea lev...,There is no evidence of sea-level rise in befo...,1 – Church et al. (2011) Sea-level rise from t...,Several social media posts have gone viral aft...
2,https://science.feedback.org/review/colorful-a...,Incorrect,"Facebook, YouTube, TikTok, X/Twitter, 2024-05-11",May 2024 auroras were caused by experiments fr...,None,The origin of auroras in Earth’s atmosphere is...,The High-frequency Active Auroral Research Pro...,The vivid lights seen around the world on May ...,"1 – McCoy et al. (2018) Haarp, a Powerful Acti...","On 10 May 2024, solar eruptions caused a sever..."
3,https://science.feedback.org/review/no-evidenc...,Misleading,"PragerU, Judith Curry, 2024-04-15",Climate scientists disagree about how much war...,None,Misleading: Climate scientists have studied na...,Scientific evidence shows that modern global w...,Climate scientists disagree about how much war...,1 – IPCC (2021). Sixth Assessment Report. 2 – ...,"On 15 April 2024, PragerU posted a short video..."
4,https://science.feedback.org/review/carbon-iso...,Incorrect,"Daily Sceptic, Chris Morrison, 2024-04-08",Human-caused carbon emissions’ effect on clima...,None,Incorrect: Scientific studies demonstrate that...,Carbon isotope ratios are actually one of the ...,“Human-caused carbon emissions’ effect on clim...,1 – Graven et al. (2020) Changes to Carbon Iso...,Carbon comes in three naturally occurring isot...
...,...,...,...,...,...,...,...,...,...,...
307,https://science.feedback.org/review/the-telegr...,-2,"The Telegraph, by Christopher Booker, on 2015-...",None,“How Arctic ice has made fools all those poor ...,None,None,None,None,Christopher Booker’s article claims that sea i...
308,https://science.feedback.org/review/the-telegr...,-1.5,"The Telegraph, by Dan Hyde, on 2015-07-11.",None,“Earth heading for 'mini ice age' within 15 ye...,None,None,None,None,The article’s central message about an imminen...
309,https://science.feedback.org/review/pope-franc...,0.9,"Vatican.va, by Pope Francis, on 2015-06-18.",None,Pope Francis’ Encyclical Laudato Si,None,None,None,None,Pope Francis’s encyclical rather accurately de...
310,https://science.feedback.org/review/mashable-a...,1.5,"Mashable, by Andrew Freedman, on 2015-03-16.",None,"""Vanuatu's president makes a leap in tying Cyc...",None,None,None,None,The main reason for the positive evaluation is...


In [59]:
for ind, i in enumerate(df['review']):
  if len(i) > 32767:
    print(ind) #These posts exceeds the cell character limit for an excel sheet (will show as empty cell if opened on excel/google sheets) --> don't open in excel and resave (will lead to loss of data)
len(df['review'][223])

119
223
229


61097

# Previous attempts to extract reviews

In [ ]:
# @title Attempt 5

import re
import pandas as pd
import csv

def getReview(soup, is_cred_score):
  # Gets all text from "Review section" - ResultSet object with 1 element (all text) is returned (indexing works because it's similar to a list)
  info = soup.find_all("div", class_="wp-block-group is-layout-constrained")

  heads = info[0].find_all("div", class_="wp-block-group alignwide has-primary-lighter-background-color has-background fractal-heading fractal-heading--boxed has-global-padding is-layout-constrained wp-block-group-is-layout-constrained")
  if heads == []:
    heads = [i for i in info[0].find_all("h2")]

  # info[0].find_all("div", class_="wp-block-group is-layout-flow wp-block-group-is-layout-flow")[1].text

  headings = []
  h = ""
  h2 = ""
  h3 = ""
  for i in heads:
    h1 = i.text.strip("\n")
    if h1 not in keys:
      continue
    if h_dict[h1] == "key_takeaway":
      h = h1
    if h1.lower() == "full claim":
      h2 = h1
    if h1.lower() == "verdict detail":
      h3 = h1
    if h1.lower() != "reviewed content" and h_dict[h1] != "unique":
      headings.append(h1)

  lines, temp, paragraphs = [], [], []
  text = soup.get_text()
  paragraphs = [i for i in re.sub("\n", "~*~", text).split("~*~") if i != "" and i != " "]

  review_content = {}
  c_ind = -2
  if not is_cred_score:
    #Getting the key takeaway and verdict detail
    if h:
      h_ind = paragraphs.index(h)
      review_content['key_takeaway'] = paragraphs[h_ind+1]
      headings.remove(h)
    if h3:
      h_ind = paragraphs.index(h3)
      review_content['verdict_detail'] = paragraphs[h_ind+1]
      headings.remove(h3)
    #Getting the claim and full claim
    c_ind = paragraphs.index("Claim:")
    review_content['claim'] = paragraphs[c_ind+1]
    if h2:
      c_ind = paragraphs.index(h2)
      review_content['full_claim'] = paragraphs[c_ind + 1]
      headings.remove(h2)


  #We now want to skip content until the second date appears (first one is when the review was published, second date is the date of claim publication)
  #"start" will be 0 for posts with credibility scores. If the post has verdicts, the starting paragraph will be 2 below the "Full Claim" heading
  start, end, heading_indices = c_ind+2, len(paragraphs)-1, []
  c = 0
  if is_cred_score:
    for h in headings:
      if h_dict[h] == "review":
        start = paragraphs.index(h)
        break
  for ind, p in enumerate(paragraphs):
    if "Science Feedback is a non-partisan, non-profit organization dedicated to science education." in p and end == len(paragraphs)-1:
      end = ind
  paragraphs = paragraphs[start:end]

  rev = ' '.join(paragraphs)
  if " REFERENCES " in rev:
    rev = rev.split(" REFERENCES ")
  review_content['review'] = rev[0]
  review_content['references'] = rev[1]

  # if len(headings) == 1:
  #   review_content[h_dict[headings[0]]] = paragraphs[1:]
  # else:
  #   for i in range(len(headings)-1):
  #     h1 = headings[i]
  #     i1 = paragraphs.index(h1)
  #     k1 = h_dict[h1]
  #     if len(headings) == 2:
  #       h2 = headings[i+1]
  #       i2 = paragraphs.index(h2)
  #       k2 = h_dict[h2]
  #       review_content[k1] = paragraphs[i1+1:i2]
  #       review_content[k2] = paragraphs[i2+1:]
  #     elif i != len(headings)-2:
  #       h2 = headings[i+1]
  #       i2 = paragraphs.index(h2)
  #       review_content[k1] = paragraphs[i1+1:i2]
  #     else:
  #       review_content[k1] = paragraphs[i1+1:]
  return review_content

h_content = {i: [] for i in vals}
for ind, sub_url in enumerate(urls):
  html = requests.get(sub_url)
  soup = BeautifulSoup(html.text, 'html.parser')
  is_cred_score = False
  #all credibility scores are between length 1 and 5 (e.g: 1, -1.75)
  if len(verdicts[ind]) < 6:
    is_cred_score = True
  review_content = getReview(soup, is_cred_score)

  used_keys = set()
  for key, value in review_content.items():
    h_content[key].append(value)
    used_keys.add(key)
  unused = list(set(vals) - used_keys)
  for i in unused:
    h_content[i].append(None)
  print(ind)
for key, value in h_content.items():
  df[key] = value

In [ ]:
# @title Atempt 4
# https://science.feedback.org/review/western-journal-op-ed-deceives-readers-with-completely-unsupported-claims-jay-lehr-tom-harris/ti
import re
import pandas as pd
def getPs(paragraphs, headings, ind):
  content = []
  next_ind = 0
  for next_ind, p in enumerate(paragraphs[ind+1:]):
    if p not in headings:
      content.append(p)
  return content, next_ind+1

def getReview(soup, is_cred_score):
  indices = {0: "Key Takeaway", 3: "Summarized Claim", 4: "Verdict Justification", 6: "Full Claim", 7: "Article Review"}
  p_dict = {}
  if is_cred_score:
    indices = {0: "Headline", 1: "Source", 3: "Article Review"}
  # Gets all text from "Review section" - ResultSet object with 1 element (all text) is returned (indexing works because it's similar to a list)
  info = soup.find_all("div", class_="wp-block-group is-layout-constrained")
  # for ind, i in enumerate(info[0]):
  #   print(ind, i)
  headings = info[0].find_all("div", class_="wp-block-group alignwide has-primary-lighter-background-color has-background fractal-heading fractal-heading--boxed has-global-padding is-layout-constrained wp-block-group-is-layout-constrained")
  if headings == []:
    headings = [i.text for i in info[0].find_all("h2")]
  rev_ind = headings.index("Reviewed content")
  #Getting rid of "Reviewed content and Key Takeaway (optional) headings"
  headings = headings[rev_ind+1:]
  lines, temp, paragraphs = [], [], []
  text = soup.get_text()
  paragraphs = [i for i in re.sub("\n", "~*~", text).split("~*~") if i != "" and i != " "]
  start, end, heading_indices = 0, len(paragraphs)-1, []
  c = 0
  for ind, p in enumerate(paragraphs):
    if re.findall("[\d]+-[\d]+-[\d]+", p):
      if c == 1:   #We want to skip content until the second date appears (first one is when the review was published, second date is the date of claim publication)
        start = ind + 1
      c += 1
    elif "Science Feedback is a non-partisan, non-profit organization dedicated to science education." in p and end == len(paragraphs)-1:
      end = ind
  paragraphs = paragraphs[start:end]
  paragraphs.remove('See our method of evaluation')
  pprint(headings)
  review_content = {}
  print(paragraphs)
  next_ind, c = 0, 0
  while len(paragraphs) > 1:
    content = []
    for ind, p in enumerate(paragraphs[next_ind:]):
      if p in headings:
        content, next_ind = getPs(paragraphs, headings, ind)
      break
    pprint(review_content)
    print(c)
    review_content[headings[c-1]] = content
    c+=1

  print(review_content)
  df = pd.DataFrame(review_content.items())
  # pprint(review_content)

for ind, sub_url in enumerate(urls):
  # if ind in [0, 10, 20, 30, 40, 50, 60, 71, 80, 90, 100]:
  if ind in [0]:
    html = requests.get(sub_url)
    soup = BeautifulSoup(html.text, 'html.parser')
    getReview(soup, True)
    break

In [ ]:
# @title Attempt 3
import re
import pandas as pd
def isClaim(h):
  if len(h) < 23 and "claim" in h.lower():
    return True
  return False
def getReview(soup, is_cred_score):
  indices = {0: "Key Takeaway", 3: "Summarized Claim", 4: "Verdict Justification", 6: "Full Claim", 7: "Article Review"}
  p_dict = {}
  if is_cred_score:
    indices = {0: "Headline", 1: "Source", 3: "Article Review"}
  # Gets all text from "Review section" - ResultSet object with 1 element (all text) is returned (indexing works because it's similar to a list)
  info = soup.find_all("div", class_="wp-block-group is-layout-constrained")
  # for ind, i in enumerate(info[0]):
  #   print(ind, i)
  headings = info[0].find_all("div", class_="wp-block-group alignwide has-primary-lighter-background-color has-background fractal-heading fractal-heading--boxed has-global-padding is-layout-constrained wp-block-group-is-layout-constrained")
  if headings == []:
    headings = [i.text for i in info[0].find_all("h2")]
  rev_ind = headings.index("Reviewed content")
  #Getting rid of "Reviewed content and Key Takeaway (optional) headings"
  headings = headings[rev_ind+1:]
  lines, temp, paragraphs = [], [], []
  text = soup.get_text()
  paragraphs = [i for i in re.sub("\n", "~*~", text).split("~*~") if i != "" and i != " "]
  start, end, heading_indices = 0, len(paragraphs)-1, []
  c = 0
  for ind, p in enumerate(paragraphs):
    if re.findall("[\d]+-[\d]+-[\d]+", p):
      if c == 1:   #We want to skip content until the second date appears (first one is when the review was published, second date is the date of claim publication)
        start = ind + 1
      c += 1
    elif "Science Feedback is a non-partisan, non-profit organization dedicated to science education." in p and end == len(paragraphs)-1:
      end = ind
  paragraphs = paragraphs[start:end]
  paragraphs.remove('See our method of evaluation')
  pprint(headings)
  review_content = {}
  heading_inds = []
  for i in headings:
    heading_inds.append(paragraphs.index(i))
  print(paragraphs)
  for i in range(len(heading_inds)-1):
    ind1, ind2 = heading_inds[i], heading_inds[i+1]
    h = headings[i]
    #if index of the first heading is not the first element of list paragraphs
    if isClaim(h):
      review_content[h] = paragraphs[ind1+1]
      if ind2 == ind1 + 2:
        continue
      else:
        review_content["lit_rivew"] = paragraphs[ind1+2:ind2]
    if i == 0 and ind1 != 0:
      review_content["lit review"] = paragraphs[:ind2]
    else:
      review_content[h] = paragraphs[ind1+1:ind2]
  #get paragraphs for final heading
  l = len(headings)-1
  final_h = headings[l]
  final_ind = heading_inds[l]

  if isClaim(final_h):
    review_content[final_h] = paragraphs[final_ind]
    if final_ind !=
    review_content[final_h] = paragraphs[final_ind+1:]
  else:
    review_content[final_h] = paragraphs[final_ind+1:]
  print(review_content)
  df = pd.DataFrame(review_content.items())
  # pprint(review_content)

  # for p in paragraphs:
  #   if
  # for ind, el in enumerate(info[0]):
  #   # print(re.sub("\n", "", el.text))
  #   for line in str(el).split("\n"):
  #     if line != "": lines.append(line)
  # for ind, line in enumerate(lines):
  #   if line in headings:
  #     paragraphs.append(temp)
  #     temp = []
  #   else:
  #     j = re.sub("\n", "", line.text)
  #     temp.append(j)
  # pprint(paragraphs)
  # headings = [re.sub("\n", "", i.text) for i in headings]
  # for i in range(len(paragraphs)):
  #   paragraphs[i] = [" ".join(p.split()) for p in paragraphs[i] if p != " "]
  #   while '' in paragraphs[i]:
  #     paragraphs[i].remove('')
  # while len(paragraphs) > len(headings):
  #   headings.insert(0, "NONE")
  # for i in range(len(headings)):
  #   print(headings[i])
    # pprint(paragraphs[i])
    # print("-"*15)
for ind, sub_url in enumerate(urls):
  # if ind in [0, 10, 20, 30, 40, 50, 60, 71, 80, 90, 100]:
  if ind in [0]:
    html = requests.get(sub_url)
    soup = BeautifulSoup(html.text, 'html.parser')
    # try:
    #   getSource(sub_url, "p", "Verd Label Source:")
    # except:
    #   getSource(sub_url, "figcaption", "Verd Label Source:")
    # getData(sub_url, "i", "", "Verdict")
    # getData(sub_url, "span", "reviewed-content__score-num", "Verdict")
    getReview(soup, True)
    break

['Verdict detail', 'Full Claim']
['Verdict detail', 'The origin of auroras in Earth’s atmosphere is well-studied and occur as solar winds and ejections of magnetized plasma from the Sun approach Earth and interact with our planet’s magnetic field. HAARP is incapable of producing the auroras that were observed because their power output is orders of magnitude less than would be necessary to do so. ', 'Full Claim', 'The vivid lights seen around the world on May 10 2024 were artificial aurora effects caused by High-frequency Active Auroral Research Program (HAARP) experiments, not by a geomagnetic storm.', 'On 10 May 2024, solar eruptions caused a several-day geomagnetic storm on Earth, causing a vivid display of colors, called auroras, which were witnessed in skies around the globe. This natural event coincided with experiments by the High-frequency Active Auroral Research Program (HAARP), which triggered dozens of social media users to claim that these experiments were responsible for t

In [ ]:
# @title Attempt 2
import re
def getReview(soup, is_cred_score):
  indices = {0: "Key Takeaway", 3: "Summarized Claim", 4: "Verdict Justification", 6: "Full Claim", 7: "Article Review"}
  p_dict = {}
  if is_cred_score:
    indices = {0: "Headline", 1: "Source", 3: "Article Review"}
  # Gets all text from "Review section" - ResultSet object with 1 element (all text) is returned (indexing works because it's similar to a list)
  info = soup.find_all("div", class_="wp-block-group is-layout-constrained")
  # for ind, i in enumerate(info[0]):
  #   print(ind, i)
  headings = info[0].find_all("div", class_="wp-block-group alignwide has-primary-lighter-background-color has-background fractal-heading fractal-heading--boxed has-global-padding is-layout-constrained wp-block-group-is-layout-constrained")
  paragraphs = []
  temp = []
  for ind, el in enumerate(info[0]):
    if el in headings:
      paragraphs.append(temp)
      # print(el)
      temp = []
    else:
      j = re.sub("\n", " ", el.text)
      temp.append(j)
  for i in range(len(paragraphs)):
    paragraphs[i] = [" ".join(p.split()) for p in paragraphs[i] if p != " "]
    while '' in paragraphs[i]:
      paragraphs[i].remove('')
  pprint(paragraphs)
for ind, sub_url in enumerate(url_list):
  if ind == 71:
    html = requests.get(sub_url)
    soup = BeautifulSoup(html.text, 'html.parser')
    getSource(sub_url, "p", "Verd Label Source:")
    getData(sub_url, "i", "", "Verdict")
    getData(sub_url, "span", "reviewed-content__score-num", "Verdict")
    getReview(soup, True)
    break

'The Daily Caller, by Chris White, on 2020-09-13.'
Verdict -1
[['Reviewed content Headline: "Wildfires Will Become Worse Thanks To '
  'Decades-Old Liberal Policies, Says Fire Expert Who Predicted Uptick In '
  'Blazes" Published in The Daily Caller, by Chris White, on 2020-09-13. -1 '
  'scientific credibility See our method of evaluation'],
 ['The article in The Daily Caller claims that land management practices '
  'enacted during the Clinton administration “paved the way for future '
  'debilitating wildfires.” The article received over 45,000 interactions on '
  'Facebook as of 28 Sept 2020, according to the social media analytics tool '
  'CrowdTangle. The article also appears on other media outlets, including The '
  'Daily Signal, The Stream, and Citizens Journal. Scientists who reviewed '
  'this article found that the article oversimplifies the complex drivers of '
  'wildfires in the western US and misleads readers by stating that land '
  'management practices enacted durin

In [ ]:
# @title Attempt 1
def getReview(soup, is_cred_score):
  indices = {0: "Key Takeaway", 3: "Summarized Claim", 4: "Verdict Justification", 6: "Full Claim", 7: "Article Review"}
  if is_cred_score:
    indices = {0: "Headline", 1: "Source", 3: "Article Review"}
  # Gets all text from "Review section" - ResultSet object with 1 element (all text) is returned (indexing works because it's similar to a list)
  info = soup.find_all("div", class_="wp-block-group is-layout-constrained")
  for el in info:
    for ind, j in enumerate(el.find_all("p")):
  #     print(ind)
  #     pprint(j.get_text())
      j = j.get_text()
      print(ind, j)
      # if ind in indices:
      #   print(indices[ind])
      #   pprint(j)
      #   print("---"*10)
#Verdict Labels (1-7, )
#info[0], the first paragraph, is the key takeaway
#info[3] is the summarized claim
#info[4] is the verdict justification
#info[6] is the full claim
#info[7:] is the article review

#Credibility Scores (8, 9)
#info[0] is headline of the article in question (which will be used as the claim)
#info[1] is publishing info (source)
#info[3:] is the article review